## My final submission consists of:<br />
    1. L0 Model, 10th LGBM Classifier, submission csv named as "FINAL_SUBMISSION1" 
    2. L2 Meta-Model Average, submission csv named as "FINAL_SUBMISSION2"
#### For getting to it quicker you can press ctrl + f and paste the submission csv name

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from scipy import stats
import datetime

from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from scipy.stats import mode,boxcox,skew
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, PowerTransformer

from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from keras.models import Model

import optuna
import gc

import os
import random
%matplotlib inline 
# Fixing Seed
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    
seed_everything()

In [7]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test.csv")
submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

In [8]:
train.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [9]:
test.head()

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333


In [10]:
tr_cols = train.columns
te_cols = test.columns
train_target = train.cancelled
common_col = list(set(tr_cols).intersection(te_cols))
print(common_col)
train = train[common_col]
train['cancelled'] = train_target.values
test = test[common_col]

['reassignment_method', 'reassigned_order', 'order_id', 'reassignment_reason', 'first_mile_distance', 'accept_time', 'lifetime_order_count', 'session_time', 'allot_time', 'undelivered_orders', 'delivered_orders', 'rider_id', 'order_time', 'last_mile_distance', 'order_date', 'alloted_orders']


In [11]:
date_columns = ['order_date', 'accept_time', 'allot_time', 'order_time']
train['accept_time'] = train['accept_time'].fillna(value=-1)
test['accept_time'] = test['accept_time'].fillna(value=-1)

In [12]:
def Accept_time_feat(row):
    if row['accept_time'] == -1:
        return 0
    return 1

In [13]:
train['Accept_time_f1'] = train.apply(lambda row: Accept_time_feat(row), axis=1)
test['Accept_time_f1'] = test.apply(lambda row: Accept_time_feat(row), axis=1)

In [14]:
def date_processing(df):
    
    df['order_time']=pd.to_datetime(df['order_time'])
    df['order_date']=pd.to_datetime(df['order_date'])
    df['allot_time']=pd.to_datetime(df['allot_time'])
    df['accept_time']=pd.to_datetime(df['accept_time'])
    
    df['order_day_of_week']=df['order_date'].dt.weekday
    df['order_time_hour']=df['order_time'].dt.hour
    df['diff_allot_and_order']=(df['allot_time'] - df['order_time'])/np.timedelta64(1,'s')
    df['diff_accept_and_allot']=(df['accept_time'] - df['allot_time'])/np.timedelta64(1,'s')
    df['diff_accept_and_order']=(df['accept_time'] - df['order_time'])/np.timedelta64(1,'s')

    return df

In [15]:
train = date_processing(train)
test = date_processing(test)
train.drop(date_columns, inplace=True, axis=1)
test.drop(date_columns, inplace=True, axis=1)

In [16]:
cat_cols = [col for col in train.select_dtypes('object').columns]
cat_cols

['reassignment_method', 'reassignment_reason']

In [17]:
train['reassigned_order'] = train['reassigned_order'].fillna(value=0)
test['reassigned_order'] = test['reassigned_order'].fillna(value=0)

In [18]:
train[cat_cols] = train[cat_cols].fillna(value="NONE")
test[cat_cols] = test[cat_cols].fillna(value="NONE")

In [19]:
def label_encode(train, test, object_cols):
    for cols in object_cols:
        Le = LabelEncoder()
        train[[cols]] = Le.fit_transform(train[[cols]])
        test[[cols]] = Le.transform(test[[cols]])
    return train, test
label_encode(train, test, cat_cols)

C:\Users\Sarkhel\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(        reassignment_method  reassigned_order  order_id  reassignment_reason  \
 0                         0               0.0    556753                    1   
 1                         0               0.0    556754                    1   
 2                         0               0.0    556755                    1   
 3                         0               0.0    556756                    1   
 4                         0               0.0    556757                    1   
 ...                     ...               ...       ...                  ...   
 449995                    0               0.0    130226                    1   
 449996                    0               0.0    130227                    1   
 449997                    0               0.0    130228                    1   
 449998                    0               0.0    130229                    1   
 449999                    0               0.0    130230                    1   
 
         first_mile_distan

In [20]:
cont_cols = ['delivered_orders', 'undelivered_orders', 'first_mile_distance', 'session_time', 'last_mile_distance',
               'alloted_orders', 'lifetime_order_count']
null_cols_same = ['undelivered_orders', 'delivered_orders', 'lifetime_order_count', 'alloted_orders']
train[null_cols_same] = train[null_cols_same].fillna(value=-1)
test[null_cols_same] = test[null_cols_same].fillna(value=-1)

In [21]:
train.head()

,reassignment_method,reassigned_order,order_id,reassignment_reason,first_mile_distance,lifetime_order_count,session_time,undelivered_orders,delivered_orders,rider_id,last_mile_distance,alloted_orders,cancelled,Accept_time_f1,order_day_of_week,order_time_hour,diff_allot_and_order,diff_accept_and_allot,diff_accept_and_order
0,0,0.0,556753,1,1.5666,621.0,NaN,0.0,46.0,11696,2.65,46.0,0,1,1,2,24.0,9.0,33.0
1,0,0.0,556754,1,2.5207,105.0,3.266667,0.0,8.0,18117,2.76,8.0,0,1,1,2,41.0,48.0,89.0
2,0,0.0,556755,1,2.2074,66.0,9.816667,0.0,1.0,18623,4.80,1.0,0,1,1,2,8.0,16.0,24.0
3,0,0.0,556756,1,2.1894,127.0,17.533333,0.0,1.0,15945,6.38,1.0,0,1,1,2,32.0,41.0,73.0
4,0,0.0,556757,1,2.7870,84.0,1.350000,0.0,34.0,17589,4.01,34.0,0,1,1,3,51.0,36.0,87.0


In [23]:
train[train['diff_accept_and_allot'] < 0].cancelled.value_counts()

1    181
0      1
Name: cancelled, dtype: int64

In [24]:
train[train['diff_accept_and_order'] < 0].cancelled.value_counts()

1    157
Name: cancelled, dtype: int64

In [25]:
train.head()

,reassignment_method,reassigned_order,order_id,reassignment_reason,first_mile_distance,lifetime_order_count,session_time,undelivered_orders,delivered_orders,rider_id,last_mile_distance,alloted_orders,cancelled,Accept_time_f1,order_day_of_week,order_time_hour,diff_allot_and_order,diff_accept_and_allot,diff_accept_and_order
0,0,0.0,556753,1,1.5666,621.0,NaN,0.0,46.0,11696,2.65,46.0,0,1,1,2,24.0,9.0,33.0
1,0,0.0,556754,1,2.5207,105.0,3.266667,0.0,8.0,18117,2.76,8.0,0,1,1,2,41.0,48.0,89.0
2,0,0.0,556755,1,2.2074,66.0,9.816667,0.0,1.0,18623,4.80,1.0,0,1,1,2,8.0,16.0,24.0
3,0,0.0,556756,1,2.1894,127.0,17.533333,0.0,1.0,15945,6.38,1.0,0,1,1,2,32.0,41.0,73.0
4,0,0.0,556757,1,2.7870,84.0,1.350000,0.0,34.0,17589,4.01,34.0,0,1,1,3,51.0,36.0,87.0


In [26]:
it_imputer = IterativeImputer(max_iter=1000)#best
train_ = pd.DataFrame(it_imputer.fit_transform(train[['session_time']]))
test_ = pd.DataFrame(it_imputer.transform(test[['session_time']]))

In [27]:
train['session_time'] = train_.values
test['session_time'] = test_.values
del train_
gc.collect()
del test_
gc.collect()

0

# *Modeling*

In [5]:
target = ['cancelled']
not_features = ['order_id','kfold', 'cancelled']
cols = list(train.columns)
features = [feat for feat in cols if feat not in not_features]

In [29]:
train["kfold"] = -1
train_targets = train[target]

In [30]:
# initialize stratified k-fold
kf = model_selection.StratifiedKFold(n_splits=20, shuffle=True, random_state=RANDOM_SEED)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=train, y=train_targets)):
    train.loc[valid_indicies, "kfold"] = fold

In [550]:
#train.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv", index=False)
#test.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv", index=False)

Let's create a dict to track the diff b/w CV score & Public lb to see if they co-relate

In [2]:
score_dict = {}

# 1. L0 Models

### 1. LGBM-CLF, cv - 0.7596915073267636, public-lb - 0.76176

In [38]:
diff = 0.76176 - 0.7596915073267636
print(diff)
score_dict.update({'L0_LGBMCLF': diff})

0.0020684926732363884


In [37]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    model = LGBMClassifier()
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid), (xtrain, ytrain)], verbose=False)
    preds_train = model.predict_proba(xtrain)[:, 1]
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    roc_t = roc_auc_score(ytrain, preds_train)
    print("TRAINING ACCURACY : " , roc_t , " VALIDATION ACCURACY: " , roc)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores), np.std(scores))#7574
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_1"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds1.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_1"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds1.csv", index=False)

TRAINING ACCURACY :  0.8708263709120287  VALIDATION ACCURACY:  0.7757480457941339
ROC-AUC is:  0.7757480457941339
0 0.7757480457941339
TRAINING ACCURACY :  0.8604905000347223  VALIDATION ACCURACY:  0.7743216484504705
ROC-AUC is:  0.7743216484504705
1 0.7743216484504705
TRAINING ACCURACY :  0.866425047662998  VALIDATION ACCURACY:  0.7571777273675617
ROC-AUC is:  0.7571777273675617
2 0.7571777273675617
TRAINING ACCURACY :  0.8621001500434712  VALIDATION ACCURACY:  0.7503078279347369
ROC-AUC is:  0.7503078279347369
3 0.7503078279347369
TRAINING ACCURACY :  0.8688915014247164  VALIDATION ACCURACY:  0.7725220906491459
ROC-AUC is:  0.7725220906491459
4 0.7725220906491459
TRAINING ACCURACY :  0.8602298217466734  VALIDATION ACCURACY:  0.7251634498712092
ROC-AUC is:  0.7251634498712092
5 0.7251634498712092
TRAINING ACCURACY :  0.8653739013206162  VALIDATION ACCURACY:  0.750258813216711
ROC-AUC is:  0.750258813216711
6 0.750258813216711
TRAINING ACCURACY :  0.8617546808484434  VALIDATION ACCURAC

### 2. LGBM-CLF

In [487]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        'boosting_type': 'gbdt',
        'lambda_l1': 0.5806043684908021,
        'lambda_l2': 7.106418698435042,
        'bagging_fraction': 0.45,
        'feature_fraction': 0.5,
        'learning_rate': 0.018,
        'max_depth': 8,
        'num_leaves': 775,
        'min_child_samples': 75,
        'min_data_in_leaf': 520,
        'max_bin': 331,
        'min_gain_to_split': 0.01084372652774491,
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=5000)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_2"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds2.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_2"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds2.csv", index=False)

ROC-AUC is:  0.7923071734919757
0 0.7923071734919757
ROC-AUC is:  0.7781616629773105
1 0.7781616629773105
ROC-AUC is:  0.7695241816566424
2 0.7695241816566424
ROC-AUC is:  0.764931786845759
3 0.764931786845759
ROC-AUC is:  0.7762625424700904
4 0.7762625424700904
ROC-AUC is:  0.733055680891961
5 0.733055680891961
ROC-AUC is:  0.7517155582018336
6 0.7517155582018336
ROC-AUC is:  0.7550339149114833
7 0.7550339149114833
ROC-AUC is:  0.7522834053393137
8 0.7522834053393137
ROC-AUC is:  0.7948746971898286
9 0.7948746971898286
ROC-AUC is:  0.7941040721152081
10 0.7941040721152081
ROC-AUC is:  0.7645617214175711
11 0.7645617214175711
ROC-AUC is:  0.7594952018122869
12 0.7594952018122869
ROC-AUC is:  0.758099531405513
13 0.758099531405513
ROC-AUC is:  0.7658579841185421
14 0.7658579841185421
ROC-AUC is:  0.8082976662964289
15 0.8082976662964289
ROC-AUC is:  0.7665466021429683
16 0.7665466021429683
ROC-AUC is:  0.7516916107649069
17 0.7516916107649069
ROC-AUC is:  0.7423948367258583
18 0.7423948

### 3. XGB-CLF

In [408]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.order_id.values.tolist()
    
    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        
        "metric": "auc",
        "boosting_type": "gbdt",
        'learning_rate': 0.01098685543556639,
        'reg_lambda': 0.0015620122636105737,
        'reg_alpha': 0.1719093206502173,
        'subsample': 0.4997703785123122,
        'colsample_bytree': 0.9017632656309243,
        'max_depth': 6,
        'min_child_weight': 34,
        'random_state':RANDOM_SEED, 
        'n_estimators':5000
    }
    
    model = XGBClassifier(**param)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_3"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds3.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_3"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds3.csv", index=False)

C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:14:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:14:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC-AUC is:  0.784625069175429
0 0.784625069175429
[19:17:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in langu

C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:17:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC-AUC is:  0.7636275594908687
1 0.7636275594908687
[19:21:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[20:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC-AUC is:  0.7567985309022722
17 0.7567985309022722
[20:17:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:17:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC-AUC is:  0.7412573334718495
18 0.7412573334718495
[20:21:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:21:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC-AUC is:  0.74635563942327
19 0.74635563942327
0.762750257560072
Wall time: 1h 10min 49s


### 4. Catboost-CLF

In [416]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.order_id.values.tolist()
    
    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
       
    param = {   
            'iterations': 260,
            'depth': 8,
            'learning_rate': 0.17572035142788606,
            'random_strength': 78,
            'bagging_temperature': 12.79633128785124,
            'od_type': 'Iter'
    }
    
    model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC",
                               random_state=RANDOM_SEED, 
                               l2_leaf_reg=50,
                                border_count=64,**param)
    
    model.fit(xtrain, ytrain, verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_4"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds4.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_4"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds4.csv", index=False)

ROC-AUC is:  0.7869268815716657
0 0.7869268815716657
ROC-AUC is:  0.7605739831211953
1 0.7605739831211953
ROC-AUC is:  0.7657256702224304
2 0.7657256702224304
ROC-AUC is:  0.7638949834254449
3 0.7638949834254449
ROC-AUC is:  0.7752450348262924
4 0.7752450348262924
ROC-AUC is:  0.7157001636178479
5 0.7157001636178479
ROC-AUC is:  0.7421877516957456
6 0.7421877516957456
ROC-AUC is:  0.7439169633823015
7 0.7439169633823015
ROC-AUC is:  0.7378344866866895
8 0.7378344866866895
ROC-AUC is:  0.790404968386799
9 0.790404968386799
ROC-AUC is:  0.7848047138203759
10 0.7848047138203759
ROC-AUC is:  0.7543008476875821
11 0.7543008476875821
ROC-AUC is:  0.752918615410882
12 0.752918615410882
ROC-AUC is:  0.7641385926039632
13 0.7641385926039632
ROC-AUC is:  0.7580755839685864
14 0.7580755839685864
ROC-AUC is:  0.804774981096169
15 0.804774981096169
ROC-AUC is:  0.7494827612049453
16 0.7494827612049453
ROC-AUC is:  0.7491089055349419
17 0.7491089055349419
ROC-AUC is:  0.7314369030692172
18 0.7314369

### 5. Naive Bias

In [418]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.order_id.values.tolist()
    
    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    model = GaussianNB()
    
    model.fit(xtrain, ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_5"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds5.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_5"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds5.csv", index=False)

ROC-AUC is:  0.7247490661317101
0 0.7247490661317101
ROC-AUC is:  0.7277185943552851
1 0.7277185943552851
ROC-AUC is:  0.687054032825906
2 0.687054032825906
ROC-AUC is:  0.7029870034330976
3 0.7029870034330976
ROC-AUC is:  0.7250956562278238
4 0.7250956562278238
ROC-AUC is:  0.7104732478702718
5 0.7104732478702718
ROC-AUC is:  0.7151095750294735
6 0.7151095750294735
ROC-AUC is:  0.6909405812404738
7 0.6909405812404738
ROC-AUC is:  0.7089909187528932
8 0.7089909187528932
ROC-AUC is:  0.7248208636961853
9 0.7248208636961853
ROC-AUC is:  0.7406623516486431
10 0.7406623516486431
ROC-AUC is:  0.6945986814437858
11 0.6945986814437858
ROC-AUC is:  0.683030863422254
12 0.683030863422254
ROC-AUC is:  0.7083644951509886
13 0.7083644951509886
ROC-AUC is:  0.6942527357362434
14 0.6942527357362434
ROC-AUC is:  0.7521171515505795
15 0.7521171515505795
ROC-AUC is:  0.7079070818773205
16 0.7079070818773205
ROC-AUC is:  0.6961583659509483
17 0.6961583659509483
ROC-AUC is:  0.6689428791607164
18 0.66894

### 6. Lgbm-Regressor

In [425]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.order_id.values.tolist()
    
    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    params_lgb = {  
                "task": "train",
                "objective": "regression",
                "metric": "auc",
                "verbosity": -1,
                "boosting_type": "gbdt",
                'learning_rate': 0.01243326765467829,
                'lambda_l1': 0.44583405781585317,
                'lambda_l2': 1.945205216342612e-07,
                'num_leaves': 135,
                'feature_fraction': 0.45977662840830263,
                'bagging_fraction': 0.5398639147101038,
                'bagging_freq': 4,
                'min_child_samples': 95,
                'n_estimators': 5000,
                'random_state':RANDOM_SEED,
             }
    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_val = lgb.Dataset(xvalid, yvalid)
    
    model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=False)
    
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    test_preds = model.predict(xtest,num_iteration=model.best_iteration)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_6"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds6.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_6"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds6.csv", index=False)

C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.78805720807969
0 0.78805720807969


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.7776337679856115
1 0.7776337679856115


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

ROC-AUC is:  0.7457989907275179
17 0.7457989907275179


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.7357534716461486
18 0.7357534716461486


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.7538952918133017
19 0.7538952918133017
0.764101942016217
Wall time: 2min 27s


### 7. LGBM - clf 2

In [448]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        'boosting_type': 'gbdt',
        'lambda_l1': 0.004120454129529359,
        'lambda_l2': 0.3054014755871697,
        'bagging_fraction': 0.6,
        'feature_fraction': 0.6,
        'learning_rate': 0.0199,
        'max_depth': 7,
        'num_leaves': 685,
        'min_child_samples': 45,
        'min_data_in_leaf': 500,
        'max_bin': 288,
        'min_gain_to_split': 0.02084372652774491,
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=5000)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_7"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds7.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_7"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds7.csv", index=False)

ROC-AUC is:  0.7912446389042612
0 0.7912446389042612
ROC-AUC is:  0.7761901632540121
1 0.7761901632540121
ROC-AUC is:  0.7645345005899856
2 0.7645345005899856
ROC-AUC is:  0.7732498170777616
3 0.7732498170777616
ROC-AUC is:  0.7782155507074916
4 0.7782155507074916
ROC-AUC is:  0.7354311288080947
5 0.7354311288080947
ROC-AUC is:  0.7492343280822978
6 0.7492343280822978
ROC-AUC is:  0.7572214012903018
7 0.7572214012903018
ROC-AUC is:  0.7505541936527578
8 0.7505541936527578
ROC-AUC is:  0.7935413934892949
9 0.7935413934892949
ROC-AUC is:  0.7912121176098255
10 0.7912121176098255
ROC-AUC is:  0.7616211829034596
11 0.7616211829034596
ROC-AUC is:  0.7591232412631911
12 0.7591232412631911
ROC-AUC is:  0.7559654529795522
13 0.7559654529795522
ROC-AUC is:  0.7604186425455678
14 0.7604186425455678
ROC-AUC is:  0.8139642500946268
15 0.8139642500946268
ROC-AUC is:  0.7685245915196095
16 0.7685245915196095
ROC-AUC is:  0.7576040434299689
17 0.7576040434299689
ROC-AUC is:  0.7414095461374939
18 0.7

### 8. CatBoost Regressor

In [441]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    valid_ids = xvalid.order_id.values.tolist()
    
    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
       
    param = {   
            'random_state':RANDOM_SEED,
            'learning_rate': 0.023897097025582358,
            'iterations': 27,
            'l2_leaf_reg': 2.328838945916324,
            'depth': 6,
    }
    
    model = CatBoostRegressor(
        **param
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_8"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds8.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_8"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds8.csv", index=False)

ROC-AUC is:  0.7314410971223022
0 0.7314410971223022
ROC-AUC is:  0.7328245192307693
1 0.7328245192307693
ROC-AUC is:  0.714832887376927
2 0.714832887376927
ROC-AUC is:  0.7009042655553678
3 0.7009042655553678
ROC-AUC is:  0.7346087324759463
4 0.7346087324759463
ROC-AUC is:  0.7036276404418111
5 0.7036276404418111
ROC-AUC is:  0.7236520048053374
6 0.7236520048053374
ROC-AUC is:  0.6892422083396001
7 0.6892422083396001
ROC-AUC is:  0.7113756699898474
8 0.7113756699898474
ROC-AUC is:  0.7428785232666577
9 0.7428785232666577
ROC-AUC is:  0.7553913174863323
10 0.7553913174863323
ROC-AUC is:  0.7181702814375146
11 0.7181702814375146
ROC-AUC is:  0.7019831062030926
12 0.7019831062030926
ROC-AUC is:  0.7327696899185943
13 0.7327696899185943
ROC-AUC is:  0.7110582372377819
14 0.7110582372377819
ROC-AUC is:  0.7609752912413197
15 0.7609752912413197
ROC-AUC is:  0.7429751744329582
16 0.7429751744329582
ROC-AUC is:  0.7141817410613608
17 0.7141817410613608
ROC-AUC is:  0.6912874745084703
18 0.691

### 9. LGBM Clf - 2

In [453]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        'boosting_type': 'gbdt',
        'lambda_l1': 2.013079655498328,
        'lambda_l2': 6.704712390093808,
        'bagging_fraction': 0.45,
        'feature_fraction': 0.6,
        'learning_rate': 0.0199,
        'max_depth': 10,
        'num_leaves': 790,
        'min_child_samples': 65,
        'min_data_in_leaf': 480,
        'max_bin': 336,
        'min_gain_to_split': 0.01084372652774491,
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=5000)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_9"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds9.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_9"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds9.csv", index=False)

ROC-AUC is:  0.7944249792473712
0 0.7944249792473712
ROC-AUC is:  0.7762674667957942
1 0.7762674667957942
ROC-AUC is:  0.768639504760113
2 0.768639504760113
ROC-AUC is:  0.7664914713529216
3 0.7664914713529216
ROC-AUC is:  0.7802786482412675
4 0.7802786482412675
ROC-AUC is:  0.7299661169610048
5 0.7299661169610048
ROC-AUC is:  0.7478705301635197
6 0.7478705301635197
ROC-AUC is:  0.7538622133392736
7 0.7538622133392736
ROC-AUC is:  0.7503417678273593
8 0.7503417678273593
ROC-AUC is:  0.794781663981625
9 0.794781663981625
ROC-AUC is:  0.7904258147167854
10 0.7904258147167854
ROC-AUC is:  0.7664563254742669
11 0.7664563254742669
ROC-AUC is:  0.7548914362759566
12 0.7548914362759566
ROC-AUC is:  0.7592991429401836
13 0.7592991429401836
ROC-AUC is:  0.7690521242668681
14 0.7690521242668681
ROC-AUC is:  0.8152119287868694
15 0.8152119287868694
ROC-AUC is:  0.7662368360163938
16 0.7662368360163938
ROC-AUC is:  0.7508687837234611
17 0.7508687837234611
ROC-AUC is:  0.7450237139924873
18 0.74502

### 10. LGBM Clf - 3, CV - 0.7688429025270969 , public-lb - 0.77032

In [3]:
diff = 0.77032 - 0.7688429025270969
print(diff)
score_dict.update({'L0_LGBMCLF_TUNED': diff})

0.0014770974729031439


In [6]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        'learning_rate': 0.010652195229674963,
        'lambda_l1': 1.919242522820974e-06,
        'lambda_l2': 0.023974154523133086,
        'num_leaves': 21,
        'feature_fraction': 0.819493399510842,
        'bagging_fraction': 0.9243051254494783,
        'bagging_freq': 3,
        'min_child_samples': 80
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=500, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_10"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds10.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_10"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds10.csv", index=False)

ROC-AUC is:  0.7864198256779191
0 0.7864198256779191
ROC-AUC is:  0.7781665052573326
1 0.7781665052573326
ROC-AUC is:  0.7716032154344159
2 0.7716032154344159
ROC-AUC is:  0.7661849786170061
3 0.7661849786170061
ROC-AUC is:  0.776917048318175
4 0.776917048318175
ROC-AUC is:  0.7359895003410357
5 0.7359895003410357
ROC-AUC is:  0.7588837668939261
6 0.7588837668939261
ROC-AUC is:  0.7565400191820693
7 0.7565400191820693
ROC-AUC is:  0.7651755683079964
8 0.7651755683079964
ROC-AUC is:  0.7971953588833535
9 0.7971953588833535
ROC-AUC is:  0.7921300452641015
10 0.7921300452641015
ROC-AUC is:  0.7675430911730609
11 0.7675430911730609
ROC-AUC is:  0.7540749837321098
12 0.7540749837321098
ROC-AUC is:  0.764575331831364
13 0.764575331831364
ROC-AUC is:  0.762831131461264
14 0.762831131461264
ROC-AUC is:  0.8077858113675899
15 0.8077858113675899
ROC-AUC is:  0.7664950893110185
16 0.7664950893110185
ROC-AUC is:  0.7522782368277467
17 0.7522782368277467
ROC-AUC is:  0.7400760701532411
18 0.7400760

In [ ]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\FINAL_SUBMISSION1.csv", index=False)
submission.head()

### 11. LGBM Clf - 4

In [461]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        'learning_rate': 0.04394129972546718,
        'lambda_l1': 3.332009828221393e-08,
        'lambda_l2': 2.8764308768784934,
        'num_leaves': 96,
        'feature_fraction': 0.5517245233128265,
        'bagging_fraction': 0.5205133833553873,
        'bagging_freq': 5,
        'min_child_samples': 27
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=500, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_11"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds11.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_11"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds11.csv", index=False)

ROC-AUC is:  0.7955257332595463
0 0.7955257332595463
ROC-AUC is:  0.7735181758439402
1 0.7735181758439402
ROC-AUC is:  0.7639365038016986
2 0.7639365038016986
ROC-AUC is:  0.7742085759734159
3 0.7742085759734159
ROC-AUC is:  0.7768722545512622
4 0.7768722545512622
ROC-AUC is:  0.7293359031172844
5 0.7293359031172844
ROC-AUC is:  0.7520318711097259
6 0.7520318711097259
ROC-AUC is:  0.7377359403994811
7 0.7377359403994811
ROC-AUC is:  0.7535131665247911
8 0.7535131665247911
ROC-AUC is:  0.7958052015555842
9 0.7958052015555842
ROC-AUC is:  0.794367149353962
10 0.794367149353962
ROC-AUC is:  0.7583045490309988
11 0.7583045490309988
ROC-AUC is:  0.7516194238866896
12 0.7516194238866896
ROC-AUC is:  0.7670572510857752
13 0.7670572510857752
ROC-AUC is:  0.7672271228326061
14 0.7672271228326061
ROC-AUC is:  0.8117772805669651
15 0.8117772805669651
ROC-AUC is:  0.7577503123073115
16 0.7577503123073115
ROC-AUC is:  0.7537357570896042
17 0.7537357570896042
ROC-AUC is:  0.7402802263601326
18 0.740

### 12. LGBM Clf - 5

In [466]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        'learning_rate': 0.033393159236481285,
        'lambda_l1': 3.346640904962391e-08,
        'lambda_l2': 3.908099023297473e-08,
        'num_leaves': 16,
        'feature_fraction': 0.43480396855094344,
        'bagging_fraction': 0.9883525790842624,
        'bagging_freq': 5,
        'min_child_samples': 25
    }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=500, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_12"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds12.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_12"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds12.csv", index=False)

ROC-AUC is:  0.7986787493082456
0 0.7986787493082456
ROC-AUC is:  0.7774878942999448
1 0.7774878942999448
ROC-AUC is:  0.7701397513842566
2 0.7701397513842566
ROC-AUC is:  0.7623244450439918
3 0.7623244450439918
ROC-AUC is:  0.7759339974181563
4 0.7759339974181563
ROC-AUC is:  0.737595873736019
5 0.737595873736019
ROC-AUC is:  0.7578536825386487
6 0.7578536825386487
ROC-AUC is:  0.7535526194964183
7 0.7535526194964183
ROC-AUC is:  0.7566731944967757
8 0.7566731944967757
ROC-AUC is:  0.8005035508535883
9 0.8005035508535883
ROC-AUC is:  0.7920215065211972
10 0.7920215065211972
ROC-AUC is:  0.7645474218689028
11 0.7645474218689028
ROC-AUC is:  0.7590319308921765
12 0.7590319308921765
ROC-AUC is:  0.7652488750303865
13 0.7652488750303865
ROC-AUC is:  0.7652246691678818
14 0.7652246691678818
ROC-AUC is:  0.8104345012618922
15 0.8104345012618922
ROC-AUC is:  0.7630501040679805
16 0.7630501040679805
ROC-AUC is:  0.7552776963737208
17 0.7552776963737208
ROC-AUC is:  0.7356699140424842
18 0.735

Getting oof preds of validation & test

In [553]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_20Folds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_20Folds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

df1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds1.csv")
df2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds2.csv")
df3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds3.csv")
df4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds4.csv")
df5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds5.csv")
df6 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds6.csv")
df7 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds7.csv")
df8 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds8.csv")
df9 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds9.csv")
df10 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds10.csv")
df11 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds11.csv")
df12 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds12.csv")

df_test1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds1.csv")
df_test2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds2.csv")
df_test3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds3.csv")
df_test4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds4.csv")
df_test5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds5.csv")
df_test6 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds6.csv")
df_test7 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds7.csv")
df_test8 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds8.csv")
df_test9 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds9.csv")
df_test10 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds10.csv")
df_test11 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds11.csv")
df_test12 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds12.csv")

train = train.merge(df1, on="order_id", how="left")
train = train.merge(df2, on="order_id", how="left")
train = train.merge(df3, on="order_id", how="left")
train = train.merge(df4, on="order_id", how="left")
train = train.merge(df5, on="order_id", how="left")
train = train.merge(df6, on="order_id", how="left")
train = train.merge(df7, on="order_id", how="left")
train = train.merge(df8, on="order_id", how="left")
train = train.merge(df9, on="order_id", how="left")
train = train.merge(df10, on="order_id", how="left")
train = train.merge(df11, on="order_id", how="left")
train = train.merge(df12, on="order_id", how="left")

test = test.merge(df_test1, on="order_id", how="left")
test = test.merge(df_test2, on="order_id", how="left")
test = test.merge(df_test3, on="order_id", how="left")
test = test.merge(df_test4, on="order_id", how="left")
test = test.merge(df_test5, on="order_id", how="left")
test = test.merge(df_test6, on="order_id", how="left")
test = test.merge(df_test7, on="order_id", how="left")
test = test.merge(df_test8, on="order_id", how="left")
test = test.merge(df_test9, on="order_id", how="left")
test = test.merge(df_test10, on="order_id", how="left")
test = test.merge(df_test11, on="order_id", how="left")
test = test.merge(df_test12, on="order_id", how="left")

train.head()

,lifetime_order_count,rider_id,first_mile_distance,last_mile_distance,alloted_orders,reassigned_order,reassignment_reason,delivered_orders,order_id,reassignment_method,...,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12
0,621.0,11696,1.5666,2.65,46.0,0.0,1,46.0,556753,0,...,0.006742,0.003334,1.116932e-13,0.005490,0.004331,0.009546,0.005139,0.004369,0.005222,0.005685
1,105.0,18117,2.5207,2.76,8.0,0.0,1,8.0,556754,0,...,0.011698,0.017587,2.632376e-13,0.015991,0.017396,0.012325,0.014709,0.017112,0.020315,0.016773
2,66.0,18623,2.2074,4.80,1.0,0.0,1,1.0,556755,0,...,0.103087,0.020978,2.805588e-13,0.091615,0.066154,0.021493,0.098316,0.081646,0.046981,0.075391
3,127.0,15945,2.1894,6.38,1.0,0.0,1,1.0,556756,0,...,0.075729,0.024250,1.928160e-13,0.065025,0.045732,0.025672,0.104228,0.065323,0.042542,0.045629
4,84.0,17589,2.7870,4.01,34.0,0.0,1,34.0,556757,0,...,0.011196,0.010543,2.393230e-13,0.017169,0.016855,0.011861,0.021343,0.017120,0.014491,0.013588


In [554]:
%%time
test.head()

Wall time: 0 ns


,lifetime_order_count,rider_id,first_mile_distance,last_mile_distance,alloted_orders,reassigned_order,reassignment_reason,delivered_orders,order_id,reassignment_method,...,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12
0,747.0,12884,1.6585,4.54,216.0,0.0,1,215.0,130231,0,...,0.004996,0.005912,1.247497e-13,0.006389,0.005208,0.009735,0.004678,0.005410,0.005261,0.005528
1,75.0,3541,2.0709,5.84,52.0,0.0,1,52.0,130232,0,...,0.005327,0.006257,6.479355e-14,0.007890,0.006427,0.009473,0.006202,0.006726,0.006044,0.006559
2,2214.0,603,1.3884,0.99,289.0,0.0,1,289.0,130233,0,...,0.003306,0.003313,4.365101e-14,0.005146,0.003792,0.009473,0.004042,0.003202,0.003977,0.003594
3,1020.0,3414,1.9039,2.59,125.0,0.0,1,122.0,130234,0,...,0.005969,0.006257,6.006810e-14,0.006662,0.005931,0.009631,0.006568,0.006064,0.005997,0.006174
4,7284.0,1426,0.8275,0.94,352.0,0.0,1,350.0,130235,0,...,0.002686,0.003180,2.506175e-15,0.005211,0.004717,0.009631,0.004417,0.004252,0.004393,0.003748


In [567]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", 'pred_6', 
                  "pred_7", "pred_9", "pred_10", "pred_11", "pred_12"]

In [563]:
#train.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv", index=False)#adding 5,8
#test.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv", index=False)#adding 5, 8

## *L1 Models*

### 1. Logistic Regression(Solver='Saga'), CV - 0.7668689554420469 , public-lb - 0.76961

In [449]:
diff = 0.76961 - 0.7668689554420469
print(diff)
score_dict.update({"L1-Model: LogR":diff})

0.0027410445579530984


In [73]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = LogisticRegression(solver='saga',random_state=RANDOM_SEED)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_1_L0"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds1_L0.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_1_L0"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds1_L0.csv", index=False)

ROC-AUC is:  0.7937605492529054
0 0.7937605492529054
ROC-AUC is:  0.773381467902601
1 0.773381467902601
ROC-AUC is:  0.7710483757177125
2 0.7710483757177125
ROC-AUC is:  0.7677207156872423
3 0.7677207156872423
ROC-AUC is:  0.7787904614774466
4 0.7787904614774466
ROC-AUC is:  0.7307291615519937
5 0.7307291615519937
ROC-AUC is:  0.7543210048826929
6 0.7543210048826929
ROC-AUC is:  0.7507910837662393
7 0.7507910837662393
ROC-AUC is:  0.7528166234492958
8 0.7528166234492958
ROC-AUC is:  0.795209272171924
9 0.795209272171924
ROC-AUC is:  0.7905517541152982
10 0.7905517541152982
ROC-AUC is:  0.7670396781464478
11 0.7670396781464478
ROC-AUC is:  0.7580972917171673
12 0.7580972917171673
ROC-AUC is:  0.7641751167523692
13 0.7641751167523692
ROC-AUC is:  0.7677620637797772
14 0.7677620637797772
ROC-AUC is:  0.8124979433631058
15 0.8124979433631058
ROC-AUC is:  0.7618344701474524
16 0.7618344701474524
ROC-AUC is:  0.7520410021468273
17 0.7520410021468273
ROC-AUC is:  0.7404308023304474
18 0.74043

In [68]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L1_Meta_Logistic.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.008331
1,130232,0.008459
2,130233,0.008109
3,130234,0.008401
4,130235,0.008105


### 2. Ridge Regression, CV - 0.7664088581022975, public-lb - 0.77221

In [450]:
diff = 0.77221 - 0.7664088581022975
print(diff)
score_dict.update({"L1-Model: RidgeR":diff})

0.005801141897702422


In [74]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = Ridge(random_state=RANDOM_SEED, alpha=1.0)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_2_L0"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds2_L0.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_2_L0"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds2_L0.csv", index=False)

ROC-AUC is:  0.7936702753182069
0 0.7936702753182069
ROC-AUC is:  0.771033653846154
1 0.771033653846154
ROC-AUC is:  0.7700026135440157
2 0.7700026135440157
ROC-AUC is:  0.7680961219107161
3 0.7680961219107161
ROC-AUC is:  0.7769556398712075
4 0.7769556398712075
ROC-AUC is:  0.733321514670217
5 0.733321514670217
ROC-AUC is:  0.7559769959887181
6 0.7559769959887181
ROC-AUC is:  0.7475383671534889
7 0.7475383671534889
ROC-AUC is:  0.7576695112431494
8 0.7576695112431494
ROC-AUC is:  0.7984073748457847
9 0.7984073748457847
ROC-AUC is:  0.794484302282811
10 0.794484302282811
ROC-AUC is:  0.7625744287201094
11 0.7625744287201094
ROC-AUC is:  0.7520470320769888
12 0.7520470320769888
ROC-AUC is:  0.7632788968466739
13 0.7632788968466739
ROC-AUC is:  0.7623957705036145
14 0.7623957705036145
ROC-AUC is:  0.8080065068114951
15 0.8080065068114951
ROC-AUC is:  0.7622948122443417
16 0.7622948122443417
ROC-AUC is:  0.7543511545334997
17 0.7543511545334997
ROC-AUC is:  0.7409092342178208
18 0.7409092

In [71]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L1_Meta_Ridge.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.005108
1,130232,0.005885
2,130233,0.002748
3,130234,0.005570
4,130235,0.002585


### 3. Lgbm Clf

In [75]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = LGBMClassifier(random_state=RANDOM_SEED)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_3_L0"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds3_L0.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_3_L0"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds3_L0.csv", index=False)

ROC-AUC is:  0.788607152739347
0 0.788607152739347
ROC-AUC is:  0.7586332664637521
1 0.7586332664637521
ROC-AUC is:  0.7641609033455603
2 0.7641609033455603
ROC-AUC is:  0.7608186853408435
3 0.7608186853408435
ROC-AUC is:  0.7711669930581722
4 0.7711669930581722
ROC-AUC is:  0.7384930412021683
5 0.7384930412021683
ROC-AUC is:  0.7641579745223391
6 0.7641579745223391
ROC-AUC is:  0.7472074962713496
7 0.7472074962713496
ROC-AUC is:  0.7388289944540147
8 0.7388289944540147
ROC-AUC is:  0.7954323795878939
9 0.7954323795878939
ROC-AUC is:  0.8068338576788318
10 0.8068338576788318
ROC-AUC is:  0.7539668756985027
11 0.7539668756985027
ROC-AUC is:  0.7463080891168548
12 0.7463080891168548
ROC-AUC is:  0.7557184842685152
13 0.7557184842685152
ROC-AUC is:  0.7587680783766877
14 0.7587680783766877
ROC-AUC is:  0.7882242872148769
15 0.7882242872148769
ROC-AUC is:  0.7620712741190746
16 0.7620712741190746
ROC-AUC is:  0.752261525307014
17 0.752261525307014
ROC-AUC is:  0.7297055378361752
18 0.72970

### 4. Linear Regression

In [85]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_4_L0"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds4_L0.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_4_L0"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds4_L0.csv", index=False)

ROC-AUC is:  0.7936605907581626
0 0.7936605907581626
ROC-AUC is:  0.7707944798007748
1 0.7707944798007748
ROC-AUC is:  0.7698918351127657
2 0.7698918351127657
ROC-AUC is:  0.7680856126038635
3 0.7680856126038635
ROC-AUC is:  0.7769835498336687
4 0.7769835498336687
ROC-AUC is:  0.7332677621499215
5 0.7332677621499215
ROC-AUC is:  0.7559320299380863
6 0.7559320299380863
ROC-AUC is:  0.7465522151465298
7 0.7465522151465298
ROC-AUC is:  0.7576097287926925
8 0.7576097287926925
ROC-AUC is:  0.7984625056358312
9 0.7984625056358312
ROC-AUC is:  0.7943988495582387
10 0.7943988495582387
ROC-AUC is:  0.7602851226634235
11 0.7602851226634235
ROC-AUC is:  0.7517074608670453
12 0.7517074608670453
ROC-AUC is:  0.7631846576524379
13 0.7631846576524379
ROC-AUC is:  0.762253119584369
14 0.762253119584369
ROC-AUC is:  0.807759107391161
15 0.807759107391161
ROC-AUC is:  0.762294467676904
16 0.762294467676904
ROC-AUC is:  0.7544447045928601
17 0.7544447045928601
ROC-AUC is:  0.7408068976887966
18 0.7408068

### 5. Lasso Regression

In [83]:
%%time
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}
for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = model = Lasso(random_state=RANDOM_SEED, normalize=True, alpha=0.000001)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "pred_5_L0"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds5_L0.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "pred_5_L0"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds5_L0.csv", index=False)

ROC-AUC is:  0.7943706765356946
0 0.7943706765356946
ROC-AUC is:  0.7744718456004427
1 0.7744718456004427
ROC-AUC is:  0.7705768351790949
2 0.7705768351790949
ROC-AUC is:  0.7666558300207481
3 0.7666558300207481
ROC-AUC is:  0.7770776167441856
4 0.7770776167441856
ROC-AUC is:  0.732814828252945
5 0.732814828252945
ROC-AUC is:  0.7553553963309424
6 0.7553553963309424
ROC-AUC is:  0.7540946240760639
7 0.7540946240760639
ROC-AUC is:  0.7568366917460077
8 0.7568366917460077
ROC-AUC is:  0.7978960367681022
9 0.7978960367681022
ROC-AUC is:  0.7938763130388281
10 0.7938763130388281
ROC-AUC is:  0.7697603826352483
11 0.7697603826352483
ROC-AUC is:  0.7556908327316323
12 0.7556908327316323
ROC-AUC is:  0.7637802424686603
13 0.7637802424686603
ROC-AUC is:  0.7672464186091225
14 0.7672464186091225
ROC-AUC is:  0.814092773748923
15 0.814092773748923
ROC-AUC is:  0.7623359880531579
16 0.7623359880531579
ROC-AUC is:  0.7531250113061191
17 0.7531250113061191
ROC-AUC is:  0.7429826687747303
18 0.74298

Getting oof preds from our L1 Models

In [86]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L0_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L0_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

df1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds1_L0.csv")
df2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds2_L0.csv")
df3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds3_L0.csv")
df4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds4_L0.csv")
df5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_preds5_L0.csv")

df_test1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds1_L0.csv")
df_test2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds2_L0.csv")
df_test3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds3_L0.csv")
df_test4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds4_L0.csv")
df_test5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_preds5_L0.csv")

train = train.merge(df1, on="order_id", how="left")
train = train.merge(df2, on="order_id", how="left")
train = train.merge(df3, on="order_id", how="left")
train = train.merge(df4, on="order_id", how="left")
train = train.merge(df5, on="order_id", how="left")

test = test.merge(df_test1, on="order_id", how="left")
test = test.merge(df_test2, on="order_id", how="left")
test = test.merge(df_test3, on="order_id", how="left")
test = test.merge(df_test4, on="order_id", how="left")
test = test.merge(df_test5, on="order_id", how="left")

train.head()

,first_mile_distance,alloted_orders,last_mile_distance,delivered_orders,reassignment_reason,lifetime_order_count,reassigned_order,rider_id,reassignment_method,order_id,...,pred_8,pred_9,pred_10,pred_11,pred_12,pred_1_L0,pred_2_L0,pred_3_L0,pred_4_L0,pred_5_L0
0,1.5666,46.0,2.65,46.0,1,621.0,0.0,11696,0,556753,...,0.009546,0.005139,0.004562,0.005222,0.005685,0.008239,0.005294,0.002894,0.005313,0.004988
1,2.5207,8.0,2.76,8.0,1,105.0,0.0,18117,0,556754,...,0.012325,0.014709,0.015499,0.020315,0.016773,0.009686,0.016451,0.016990,0.016478,0.016298
2,2.2074,1.0,4.80,1.0,1,66.0,0.0,18623,0,556755,...,0.021493,0.098316,0.068902,0.046981,0.075391,0.023031,0.072058,0.039929,0.071940,0.076832
3,2.1894,1.0,6.38,1.0,1,127.0,0.0,15945,0,556756,...,0.025672,0.104228,0.046500,0.042542,0.045629,0.018851,0.063044,0.063886,0.065463,0.050825
4,2.7870,34.0,4.01,34.0,1,84.0,0.0,17589,0,556757,...,0.011861,0.021343,0.014832,0.014491,0.013588,0.009488,0.011165,0.014431,0.011160,0.012369


In [87]:
%%time
test.head()

Wall time: 0 ns


,first_mile_distance,alloted_orders,last_mile_distance,delivered_orders,reassignment_reason,lifetime_order_count,reassigned_order,rider_id,reassignment_method,order_id,...,pred_8,pred_9,pred_10,pred_11,pred_12,pred_1_L0,pred_2_L0,pred_3_L0,pred_4_L0,pred_5_L0
0,1.6585,216.0,4.54,215.0,1,747.0,0.0,12884,0,130231,...,0.009735,0.004678,0.005262,0.005261,0.005528,0.008331,0.005108,0.003775,0.005117,0.004883
1,2.0709,52.0,5.84,52.0,1,75.0,0.0,3541,0,130232,...,0.009473,0.006202,0.006746,0.006044,0.006559,0.008459,0.005885,0.005881,0.005879,0.005896
2,1.3884,289.0,0.99,289.0,1,2214.0,0.0,603,0,130233,...,0.009473,0.004042,0.003182,0.003977,0.003594,0.008109,0.002748,0.002598,0.002756,0.002663
3,1.9039,125.0,2.59,122.0,1,1020.0,0.0,3414,0,130234,...,0.009631,0.006568,0.006062,0.005997,0.006174,0.008401,0.005570,0.005400,0.005561,0.005720
4,0.8275,352.0,0.94,350.0,1,7284.0,0.0,1426,0,130235,...,0.009631,0.004417,0.004424,0.004393,0.003748,0.008105,0.002585,0.002282,0.002576,0.002690


In [88]:
train.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv", index=False)
test.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv", index=False)

## *L2 model*

In [89]:
useful_features = ['pred_1_L0', 'pred_2_L0', 'pred_3_L0', 'pred_4_L0', 'pred_5_L0']

### 1. Logistic Regression(solver='saga'), CV - 0.7670144085293248, public-lb - 0.77147

In [451]:
diff = 0.77147 - 0.7670144085293248
print(diff)
score_dict.update({"L2-Model: LogR":diff})

0.004455591470675202


In [116]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}

for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = LogisticRegression(solver="saga", random_state = RANDOM_SEED)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "oof_pred_LogR"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLogR.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "oof_pred_LogR"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Logistic.csv", index=False)

ROC-AUC is:  0.7925811081903708
0 0.7925811081903708
ROC-AUC is:  0.7724360127282789
1 0.7724360127282789
ROC-AUC is:  0.7687408075868236
2 0.7687408075868236
ROC-AUC is:  0.7689515105750331
3 0.7689515105750331
ROC-AUC is:  0.776048393807503
4 0.776048393807503
ROC-AUC is:  0.7370356070821702
5 0.7370356070821702
ROC-AUC is:  0.7577289491261683
6 0.7577289491261683
ROC-AUC is:  0.7491045984419695
7 0.7491045984419695
ROC-AUC is:  0.7552117117093836
8 0.7552117117093836
ROC-AUC is:  0.7987307513861518
9 0.7987307513861518
ROC-AUC is:  0.7976973936402155
10 0.7976973936402155
ROC-AUC is:  0.768007051228047
11 0.768007051228047
ROC-AUC is:  0.7514664359443103
12 0.7514664359443103
ROC-AUC is:  0.7630306360077451
13 0.7630306360077451
ROC-AUC is:  0.7636555090561798
14 0.7636555090561798
ROC-AUC is:  0.8081546708097456
15 0.8081546708097456
ROC-AUC is:  0.7630285686031185
16 0.7630285686031185
ROC-AUC is:  0.7533527703824991
17 0.7533527703824991
ROC-AUC is:  0.7403784280799031
18 0.74037

In [96]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Meta_Logistic.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.007901
1,130232,0.008048
2,130233,0.007559
3,130234,0.008006
4,130235,0.007542


### 2. Ridge Regression, CV - 0.7665423103311333, public lb - 0.77182

In [452]:
diff = 0.77182 - 0.7665423103311333
print(diff)
score_dict.update({"L2-Model: RidgeR":diff})

0.005277689668866659


In [161]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}

for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = Ridge(alpha=0.01, random_state = RANDOM_SEED)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "oof_pred_Ridge"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredRidge.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "oof_pred_Ridge"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Ridge.csv", index=False)

ROC-AUC is:  0.7923140910348644
0 0.7923140910348644
ROC-AUC is:  0.7716230285002766
1 0.7716230285002766
ROC-AUC is:  0.7687807774096074
2 0.7687807774096074
ROC-AUC is:  0.7692810893292805
3 0.7692810893292805
ROC-AUC is:  0.7761784680152692
4 0.7761784680152692
ROC-AUC is:  0.7366276392358253
5 0.7366276392358253
ROC-AUC is:  0.7573177078891644
6 0.7573177078891644
ROC-AUC is:  0.746449706333787
7 0.746449706333787
ROC-AUC is:  0.7557811094003338
8 0.7557811094003338
ROC-AUC is:  0.798470947538057
9 0.798470947538057
ROC-AUC is:  0.7968464843525898
10 0.7968464843525898
ROC-AUC is:  0.7661245070316738
11 0.7661245070316738
ROC-AUC is:  0.7506108749962744
12 0.7506108749962744
ROC-AUC is:  0.763569367196732
13 0.763569367196732
ROC-AUC is:  0.7631002386301791
14 0.7631002386301791
ROC-AUC is:  0.806238014437031
15 0.806238014437031
ROC-AUC is:  0.7628912584791586
16 0.7628912584791586
ROC-AUC is:  0.7534973164226526
17 0.7534973164226526
ROC-AUC is:  0.7402001144308461
18 0.740200114

In [98]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Meta_Ridge.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.004834
1,130232,0.005735
2,130233,0.002445
3,130234,0.005407
4,130235,0.002291


### 3. Lasso Regression, CV - 0.7665840725893249, public-lb - 0.77187

In [453]:
diff = 0.77187 - 0.7665840725893249
print(diff)
score_dict.update({"L2-Model: LassoR":diff})

0.005285927410675018


In [118]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}

for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = Lasso(random_state=RANDOM_SEED, normalize=True, alpha=0.000001)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "oof_pred_Lasso"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLasso.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "oof_pred_Lasso"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Lasso.csv", index=False)

ROC-AUC is:  0.7924448325954621
0 0.7924448325954621
ROC-AUC is:  0.7715414014941894
1 0.7715414014941894
ROC-AUC is:  0.7688527920041058
2 0.7688527920041058
ROC-AUC is:  0.7692664452131744
3 0.7692664452131744
ROC-AUC is:  0.7763367967529343
4 0.7763367967529343
ROC-AUC is:  0.7364736175911325
5 0.7364736175911325
ROC-AUC is:  0.757021207608945
6 0.757021207608945
ROC-AUC is:  0.7470802647449452
7 0.7470802647449452
ROC-AUC is:  0.7558414087019473
8 0.7558414087019473
ROC-AUC is:  0.7985164304398454
9 0.7985164304398454
ROC-AUC is:  0.7964431681666548
10 0.7964431681666548
ROC-AUC is:  0.7657327338549049
11 0.7657327338549049
ROC-AUC is:  0.7507759227989764
12 0.7507759227989764
ROC-AUC is:  0.7636182957728983
13 0.7636182957728983
ROC-AUC is:  0.7630337371146853
14 0.7630337371146853
ROC-AUC is:  0.8067650303331328
15 0.8067650303331328
ROC-AUC is:  0.7628848839815594
16 0.7628848839815594
ROC-AUC is:  0.7536418624628061
17 0.7536418624628061
ROC-AUC is:  0.7404285626421017
18 0.740

In [110]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Meta_Lasso.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.005088
1,130232,0.005931
2,130233,0.002815
3,130234,0.005621
4,130235,0.002660


### 4. HistGradient Boosting Clf, CV - 0.7673018912093903, publoc-lb - 0.76878

In [454]:
diff = 0.76878 - 0.7673018912093903
print(diff)
score_dict.update({"L2-Model: HistGBREG":diff})

0.0014781087906097223


In [130]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}

for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = HistGradientBoostingRegressor(random_state=RANDOM_SEED,max_bins=255, max_iter=10000)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "oof_pred_HistGrReg"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredHistGrReg.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "oof_pred_HistGrReg"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-HistGrReg.csv", index=False)

ROC-AUC is:  0.7946446977033759
0 0.7946446977033759
ROC-AUC is:  0.7673317307692308
1 0.7673317307692308
ROC-AUC is:  0.7703524356352333
2 0.7703524356352333
ROC-AUC is:  0.7656811348810959
3 0.7656811348810959
ROC-AUC is:  0.77556212301092
4 0.77556212301092
ROC-AUC is:  0.7387260549319746
5 0.7387260549319746
ROC-AUC is:  0.753637038518677
6 0.753637038518677
ROC-AUC is:  0.7530113901935074
7 0.7530113901935074
ROC-AUC is:  0.7568929423802271
8 0.7568929423802271
ROC-AUC is:  0.7984193485642477
9 0.7984193485642477
ROC-AUC is:  0.7992399359173479
10 0.7992399359173479
ROC-AUC is:  0.7717162335540116
11 0.7717162335540116
ROC-AUC is:  0.7485029836955857
12 0.7485029836955857
ROC-AUC is:  0.7632994847510819
13 0.7632994847510819
ROC-AUC is:  0.7687445116867798
14 0.7687445116867798
ROC-AUC is:  0.8105484669419416
15 0.8105484669419416
ROC-AUC is:  0.7645599124385226
16 0.7645599124385226
ROC-AUC is:  0.7565352813797995
17 0.7565352813797995
ROC-AUC is:  0.7370574009726105
18 0.7370574

In [131]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Meta_HistGrbReg.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.004797
1,130232,0.006382
2,130233,0.004282
3,130234,0.006529
4,130235,0.004287


### 5. Linear Regression, CV - 0.7664936721034142 , public-lb - 0.77190

In [455]:
diff = 0.77190 - 0.7664936721034142
print(diff)
score_dict.update({"L2-Model: LinearREG":diff})

0.005406327896585794


In [147]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

scores = []
final_test_predictions = []
final_valid_predictions = {}

for fold in range(20):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    valid_ids = xvalid.order_id.values.tolist()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["order_id", "oof_pred_LinearReg"]
final_valid_predictions.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLinearReg.csv", index=False)

sample_submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["order_id", "oof_pred_LinearReg"]
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-LinearReg.csv", index=False)

ROC-AUC is:  0.7920180201992252
0 0.7920180201992252
ROC-AUC is:  0.7715872302158273
1 0.7715872302158273
ROC-AUC is:  0.768878806845659
2 0.768878806845659
ROC-AUC is:  0.7693403549285804
3 0.7693403549285804
ROC-AUC is:  0.7759148739253586
4 0.7759148739253586
ROC-AUC is:  0.7366324631799542
5 0.7366324631799542
ROC-AUC is:  0.7575592496630562
6 0.7575592496630562
ROC-AUC is:  0.7466879747170195
7 0.7466879747170195
ROC-AUC is:  0.7560936320664104
8 0.7560936320664104
ROC-AUC is:  0.7982502520941517
9 0.7982502520941517
ROC-AUC is:  0.7969998168624068
10 0.7969998168624068
ROC-AUC is:  0.7667674698705925
11 0.7667674698705925
ROC-AUC is:  0.7507035636370403
12 0.7507035636370403
ROC-AUC is:  0.7638581147096013
13 0.7638581147096013
ROC-AUC is:  0.763022710956676
14 0.763022710956676
ROC-AUC is:  0.8049057444388108
15 0.8049057444388108
ROC-AUC is:  0.7627229372857975
16 0.7627229372857975
ROC-AUC is:  0.7532609431603278
17 0.7532609431603278
ROC-AUC is:  0.7398531350209901
18 0.73985

In [148]:
submission.cancelled = np.mean(np.column_stack(final_test_predictions), axis=1)
submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Meta_LinearReg.csv", index=False)
submission.head()

,order_id,cancelled
0,130231,0.004845
1,130232,0.005751
2,130233,0.002427
3,130234,0.005410
4,130235,0.002296


## *Blend of all oofpreds from L2 Meta models*

In [132]:
train = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_L1_preds.csv")
test = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_L1_preds.csv")
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

df1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLogR.csv")
df2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredRidge.csv")
df3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLasso.csv")
df4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredHistGrReg.csv")
df5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_oofpredLinearReg.csv")

df_test1 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Logistic.csv")
df_test2 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Ridge.csv")
df_test3 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-Lasso.csv")
df_test4 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-HistGrReg.csv")
df_test5 = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\Stack-Meta-LinearReg.csv")

train = train.merge(df1, on="order_id", how="left")
train = train.merge(df2, on="order_id", how="left")
train = train.merge(df3, on="order_id", how="left")
train = train.merge(df4, on="order_id", how="left")
train = train.merge(df5, on="order_id", how="left")

test = test.merge(df_test1, on="order_id", how="left")
test = test.merge(df_test2, on="order_id", how="left")
test = test.merge(df_test3, on="order_id", how="left")
test = test.merge(df_test4, on="order_id", how="left")
test = test.merge(df_test5, on="order_id", how="left")

train.head()

,first_mile_distance,alloted_orders,last_mile_distance,delivered_orders,reassignment_reason,lifetime_order_count,reassigned_order,rider_id,reassignment_method,order_id,...,pred_1_L0,pred_2_L0,pred_3_L0,pred_4_L0,pred_5_L0,oof_pred_LogR,oof_pred_Ridge,oof_pred_Lasso,oof_pred_HistGrReg,oof_pred_LinearReg
0,1.5666,46.0,2.65,46.0,1,621.0,0.0,11696,0,556753,...,0.008239,0.005294,0.002894,0.005313,0.004988,0.007928,0.004960,0.005217,0.004920,0.004953
1,2.5207,8.0,2.76,8.0,1,105.0,0.0,18117,0,556754,...,0.009686,0.016451,0.016990,0.016478,0.016298,0.009886,0.016812,0.016401,0.015522,0.016806
2,2.2074,1.0,4.80,1.0,1,66.0,0.0,18623,0,556755,...,0.023031,0.072058,0.039929,0.071940,0.076832,0.027269,0.072352,0.069943,0.012691,0.071991
3,2.1894,1.0,6.38,1.0,1,127.0,0.0,15945,0,556756,...,0.018851,0.063044,0.063886,0.065463,0.050825,0.022261,0.066671,0.062545,0.046970,0.065904
4,2.7870,34.0,4.01,34.0,1,84.0,0.0,17589,0,556757,...,0.009488,0.011165,0.014431,0.011160,0.012369,0.009035,0.011436,0.011386,0.013012,0.011253


In [133]:
%%time
test.head()

Wall time: 0 ns


,first_mile_distance,alloted_orders,last_mile_distance,delivered_orders,reassignment_reason,lifetime_order_count,reassigned_order,rider_id,reassignment_method,order_id,...,pred_1_L0,pred_2_L0,pred_3_L0,pred_4_L0,pred_5_L0,oof_pred_LogR,oof_pred_Ridge,oof_pred_Lasso,oof_pred_HistGrReg,oof_pred_LinearReg
0,1.6585,216.0,4.54,215.0,1,747.0,0.0,12884,0,130231,...,0.008331,0.005108,0.003775,0.005117,0.004883,0.007901,0.004834,0.005088,0.004797,0.004845
1,2.0709,52.0,5.84,52.0,1,75.0,0.0,3541,0,130232,...,0.008459,0.005885,0.005881,0.005879,0.005896,0.008048,0.005735,0.005931,0.006382,0.005751
2,1.3884,289.0,0.99,289.0,1,2214.0,0.0,603,0,130233,...,0.008109,0.002748,0.002598,0.002756,0.002663,0.007559,0.002445,0.002815,0.004282,0.002427
3,1.9039,125.0,2.59,122.0,1,1020.0,0.0,3414,0,130234,...,0.008401,0.005570,0.005400,0.005561,0.005720,0.008006,0.005407,0.005621,0.006529,0.005410
4,0.8275,352.0,0.94,350.0,1,7284.0,0.0,1426,0,130235,...,0.008105,0.002585,0.002282,0.002576,0.002690,0.007542,0.002291,0.002660,0.004287,0.002296


In [134]:
train.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\train_ALL_PREDS.csv", index=False)
test.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\test_ALL_PREDS.csv", index=False)

In [135]:
train.columns

Index(['first_mile_distance', 'alloted_orders', 'last_mile_distance',
       'delivered_orders', 'reassignment_reason', 'lifetime_order_count',
       'reassigned_order', 'rider_id', 'reassignment_method', 'order_id',
       'undelivered_orders', 'session_time', 'cancelled', 'Accept_time_f1',
       'order_day_of_week', 'order_time_hour', 'diff_allot_and_order',
       'diff_accept_and_allot', 'diff_accept_and_order', 'kfold', 'pred_1',
       'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'pred_8',
       'pred_9', 'pred_10', 'pred_11', 'pred_12', 'pred_1_L0', 'pred_2_L0',
       'pred_3_L0', 'pred_4_L0', 'pred_5_L0', 'oof_pred_LogR',
       'oof_pred_Ridge', 'oof_pred_Lasso', 'oof_pred_HistGrReg',
       'oof_pred_LinearReg'],
      dtype='object')

In [136]:
#Train oof meta - preds
tr_l2_log = train.oof_pred_LogR
tr_l2_rr = train.oof_pred_Ridge
tr_l2_lassor = train.oof_pred_Lasso
tr_l2_HistReg = train.oof_pred_HistGrReg
tr_l2_LR = train.oof_pred_LinearReg

In [138]:
#Test oof meta-preds
te_l2_log = test.oof_pred_LogR
te_l2_rr = test.oof_pred_Ridge
te_l2_lassor = test.oof_pred_Lasso
te_l2_HistReg = test.oof_pred_HistGrReg
te_l2_LR = test.oof_pred_LinearReg

### *Averaging L2 Meta*, oof-score - 0.7664323750634199 , public lb - 0.77155

In [456]:
diff = 0.77155 - 0.7664323750634199
print(diff)
score_dict.update({"Average L2":diff})

0.0051176249365800786


In [139]:
y_train = train.cancelled

In [140]:
#Checking OOF score
oof_avg = (tr_l2_log + tr_l2_rr + tr_l2_lassor + tr_l2_HistReg + tr_l2_LR)/5
print('OOF score: {}'.format(roc_auc_score(y_train.values, oof_avg)))

OOF score: 0.7664323750634199


In [144]:
sub_avg = (te_l2_log + te_l2_rr + te_l2_lassor + te_l2_HistReg + te_l2_LR)/5
sample_submission.cancelled = sub_avg
sample_submission.head()

,order_id,cancelled
0,130231,0.005493
1,130232,0.006369
2,130233,0.003906
3,130234,0.006195
4,130235,0.003815


In [146]:
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\FINAL_SUBMISSION2.csv", index=False)
sample_submission.head()

,order_id,cancelled
0,130231,0.005493
1,130232,0.006369
2,130233,0.003906
3,130234,0.006195
4,130235,0.003815


### *Weighted Blending L2 Meta*, OOF score - 0.7666002003119106 , public-lb = 0.77119

In [457]:
diff = 0.77119- 0.7666002003119106
print(diff)
score_dict.update({"Average Meta Blend":diff})

0.00458979968808948


###### *Optuna for Weights*

In [149]:
def run(trial):
    
    samples = [0.2]*5

    samples[0]=trial.suggest_uniform('w0', 0, 1)
    samples[1]=trial.suggest_uniform('w1', 0, 1)
    samples[2]=trial.suggest_uniform('w2', 0, 1)
    samples[3]=trial.suggest_uniform('w3', 0, 1)
    samples[4]=trial.suggest_uniform('w4', 0, 1)
    
    w = [samples[i]/sum(samples) for i in range(len(samples))]
    
    oof_preds = (w[0] * tr_l2_log + w[1] * tr_l2_rr + w[2] * tr_l2_lassor + w[3] * tr_l2_HistReg + w[4] * tr_l2_LR)
    roc = roc_auc_score(y_train.values, oof_preds)
    return roc

In [150]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=1250)

[I 2022-02-04 13:17:39,834] A new study created in memory with name: no-name-f1c6ed3a-283e-475c-9854-407cfdf921fb
[I 2022-02-04 13:17:39,965] Trial 0 finished with value: 0.7664430925846355 and parameters: {'w0': 0.040116027813093424, 'w1': 0.9370626596975528, 'w2': 0.08930920618465532, 'w3': 0.7939339864116544, 'w4': 0.3389560229664027}. Best is trial 0 with value: 0.7664430925846355.
[I 2022-02-04 13:17:40,094] Trial 1 finished with value: 0.766445352079741 and parameters: {'w0': 0.06651317418997182, 'w1': 0.969649516280526, 'w2': 0.8495635600796606, 'w3': 0.8764000060252303, 'w4': 0.4154568117222365}. Best is trial 1 with value: 0.766445352079741.
[I 2022-02-04 13:17:40,228] Trial 2 finished with value: 0.7664188482538582 and parameters: {'w0': 0.0409241624711042, 'w1': 0.9532959283075831, 'w2': 0.37119197311151597, 'w3': 0.6855817054382176, 'w4': 0.5922169048405301}. Best is trial 1 with value: 0.766445352079741.
[I 2022-02-04 13:17:40,356] Trial 3 finished with value: 0.7662891642

[I 2022-02-04 13:17:43,919] Trial 29 finished with value: 0.766478361648682 and parameters: {'w0': 0.7169006997975865, 'w1': 0.005815477275489234, 'w2': 0.09381760159093733, 'w3': 0.23126350337594553, 'w4': 0.3181849323342568}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:44,067] Trial 30 finished with value: 0.7664682732012371 and parameters: {'w0': 0.5527296682996293, 'w1': 0.4964592901054661, 'w2': 0.38523686358197556, 'w3': 0.4128174921089028, 'w4': 0.07141861913520564}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:44,215] Trial 31 finished with value: 0.7665041196343613 and parameters: {'w0': 0.8252497415281165, 'w1': 0.006607678750772918, 'w2': 0.04577097663427409, 'w3': 0.09555546800238328, 'w4': 0.14625927276792178}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:44,362] Trial 32 finished with value: 0.7664289603992874 and parameters: {'w0': 0.929875941684394, 'w1': 0.08345256529596617, 'w2': 0.09167635025440

[I 2022-02-04 13:17:47,977] Trial 58 finished with value: 0.7665029545552677 and parameters: {'w0': 0.6639765524174821, 'w1': 0.08354623404789407, 'w2': 0.19212650227343975, 'w3': 0.34200119238999827, 'w4': 0.04975234904418645}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:48,115] Trial 59 finished with value: 0.7665170785540395 and parameters: {'w0': 0.9208551050384838, 'w1': 0.036368592211823564, 'w2': 0.06931722822919689, 'w3': 0.27056692967891066, 'w4': 0.1701530138950172}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:48,248] Trial 60 finished with value: 0.7664781979171527 and parameters: {'w0': 0.915883532242225, 'w1': 0.036003319588886265, 'w2': 0.0737877123650868, 'w3': 0.6009173057848096, 'w4': 0.29779864769191133}. Best is trial 22 with value: 0.7665275702980934.
[I 2022-02-04 13:17:48,388] Trial 61 finished with value: 0.7665103974458957 and parameters: {'w0': 0.8666687092321222, 'w1': 0.030565181378736705, 'w2': 0.024242885622

[I 2022-02-04 13:17:51,947] Trial 87 finished with value: 0.7665360472825911 and parameters: {'w0': 0.9504892955560603, 'w1': 0.020774435945686426, 'w2': 0.1313469993040955, 'w3': 0.17328934445106872, 'w4': 0.07957599815984442}. Best is trial 87 with value: 0.7665360472825911.
[I 2022-02-04 13:17:52,084] Trial 88 finished with value: 0.7665250789936121 and parameters: {'w0': 0.9504889130116199, 'w1': 0.06629211632264875, 'w2': 0.17312913565823101, 'w3': 0.17602214277241307, 'w4': 0.04627339697643117}. Best is trial 87 with value: 0.7665360472825911.
[I 2022-02-04 13:17:52,220] Trial 89 finished with value: 0.7665218655469117 and parameters: {'w0': 0.976557921906553, 'w1': 0.004894048118901567, 'w2': 0.25061606955486987, 'w3': 0.1683578679602863, 'w4': 0.07900531447279628}. Best is trial 87 with value: 0.7665360472825911.
[I 2022-02-04 13:17:52,353] Trial 90 finished with value: 0.7664161057507409 and parameters: {'w0': 0.2978267937481237, 'w1': 0.06513765479825655, 'w2': 0.122840742002

[I 2022-02-04 13:17:55,909] Trial 116 finished with value: 0.766536920230166 and parameters: {'w0': 0.9449954367721992, 'w1': 0.0021980949963682113, 'w2': 0.15346714456922098, 'w3': 0.24215075873672726, 'w4': 0.03041653088354688}. Best is trial 92 with value: 0.7665442058523512.
[I 2022-02-04 13:17:56,046] Trial 117 finished with value: 0.7665346529793566 and parameters: {'w0': 0.9995795148442153, 'w1': 0.0030132084262076003, 'w2': 0.08424633618458911, 'w3': 0.22779949716641867, 'w4': 0.01564596124504397}. Best is trial 92 with value: 0.7665442058523512.
[I 2022-02-04 13:17:56,181] Trial 118 finished with value: 0.7665598525543459 and parameters: {'w0': 0.9486736593091251, 'w1': 0.005164714932448994, 'w2': 0.08466972114449386, 'w3': 0.15321197563452976, 'w4': 0.028439016051325948}. Best is trial 118 with value: 0.7665598525543459.
[I 2022-02-04 13:17:56,315] Trial 119 finished with value: 0.7665622576843382 and parameters: {'w0': 0.9389405242074013, 'w1': 0.003117646207728062, 'w2': 0.

...

[I 2022-02-04 13:20:25,292] Trial 1124 finished with value: 0.766524984201674 and parameters: {'w0': 0.8050394464503696, 'w1': 0.08935741154540124, 'w2': 0.0002063249374205621, 'w3': 0.15553621740131435, 'w4': 0.06442044997059483}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:25,448] Trial 1125 finished with value: 0.7665577361088924 and parameters: {'w0': 0.8326932182974462, 'w1': 0.029492510869159452, 'w2': 0.06044228871949727, 'w3': 0.07235043430638778, 'w4': 0.0006482146357663544}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:25,603] Trial 1126 finished with value: 0.7665157536212689 and parameters: {'w0': 0.8686109004712131, 'w1': 0.0004543832505978268, 'w2': 0.2427865296980995, 'w3': 0.09114912063187716, 'w4': 0.017961576254998765}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:25,764] Trial 1127 finished with value: 0.7665836733375091 and parameters: {'w0': 0.8397026978222866, 'w1': 0.000136201721549183

[I 2022-02-04 13:20:29,844] Trial 1152 finished with value: 0.7665577903988207 and parameters: {'w0': 0.8518345334740489, 'w1': 0.016465333542408943, 'w2': 0.01665653803705709, 'w3': 0.09485024326054951, 'w4': 0.04642226195382913}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:30,007] Trial 1153 finished with value: 0.7665761808965501 and parameters: {'w0': 0.8659556386102818, 'w1': 0.0007046160361064355, 'w2': 0.0397285882011348, 'w3': 0.11831244929377749, 'w4': 0.00042082879270218757}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:30,166] Trial 1154 finished with value: 0.7665783244869675 and parameters: {'w0': 0.8346577437521384, 'w1': 0.03423966432908002, 'w2': 0.0007621841846633917, 'w3': 0.07698156995015122, 'w4': 1.5452337111323557e-05}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:30,322] Trial 1155 finished with value: 0.7663941678801658 and parameters: {'w0': 0.8705762160648186, 'w1': 0.02086180718269

[I 2022-02-04 13:20:34,296] Trial 1180 finished with value: 0.7664007417010705 and parameters: {'w0': 0.8435055058421861, 'w1': 0.32666745660044816, 'w2': 0.030500028938762067, 'w3': 0.0675762213619223, 'w4': 0.03624287626632708}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:34,454] Trial 1181 finished with value: 0.7665412397264346 and parameters: {'w0': 0.8107539073509215, 'w1': 0.038723111032003696, 'w2': 0.01837216948364893, 'w3': 0.1455370359330109, 'w4': 0.01763091718169366}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:34,612] Trial 1182 finished with value: 0.7664237675245712 and parameters: {'w0': 0.8700176360227534, 'w1': 0.41242758597163304, 'w2': 0.04706400359575623, 'w3': 0.12143361615941595, 'w4': 0.05744287889789569}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:34,772] Trial 1183 finished with value: 0.7665889338591994 and parameters: {'w0': 0.8322616095630013, 'w1': 0.01695808573927225, 'w2':

[I 2022-02-04 13:20:38,766] Trial 1208 finished with value: 0.7665569928539236 and parameters: {'w0': 0.8020669457283156, 'w1': 0.020009721063749857, 'w2': 0.01834880542205015, 'w3': 0.0820890908198086, 'w4': 0.03683839496285056}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:38,927] Trial 1209 finished with value: 0.7665535816367708 and parameters: {'w0': 0.84634710455134, 'w1': 0.03237674007484137, 'w2': 0.06779134542732916, 'w3': 0.1134381968990554, 'w4': 0.019549216387915794}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:39,086] Trial 1210 finished with value: 0.7663848804245976 and parameters: {'w0': 0.8221118380636595, 'w1': 0.018490064850971204, 'w2': 0.26774014455626866, 'w3': 0.02223280422411615, 'w4': 0.018071742001548298}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:39,244] Trial 1211 finished with value: 0.7665337666747356 and parameters: {'w0': 0.8596463254150848, 'w1': 0.04823222848322917, 'w2':

[I 2022-02-04 13:20:43,400] Trial 1236 finished with value: 0.7665535570770412 and parameters: {'w0': 0.8394989668491387, 'w1': 3.6091311674332966e-05, 'w2': 0.03175670266457446, 'w3': 0.04407277516447674, 'w4': 0.03546728923685548}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:43,557] Trial 1237 finished with value: 0.7663727319759899 and parameters: {'w0': 0.8890793556878573, 'w1': 0.17015423711668692, 'w2': 0.01658056865701167, 'w3': 0.015548093062181509, 'w4': 0.018648416414385242}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:43,721] Trial 1238 finished with value: 0.7665638816425862 and parameters: {'w0': 0.820813216954248, 'w1': 0.01946265135857179, 'w2': 0.059195429883945994, 'w3': 0.0691870752802783, 'w4': 6.914365506598087e-07}. Best is trial 1034 with value: 0.7666002003119106.
[I 2022-02-04 13:20:43,881] Trial 1239 finished with value: 0.7665149802052287 and parameters: {'w0': 0.857624612904231, 'w1': 0.05035546657925373, 

In [151]:
study.best_params

{'w0': 0.8535318456738976,
 'w1': 0.0005804260245392991,
 'w2': 0.0005338456556189634,
 'w3': 0.03947761812798651,
 'w4': 0.00029262889516829937}

In [152]:
study.best_value

0.7666002003119106

In [153]:
#Checking OOF score
oof_blend = (0.8535318456738976 * tr_l2_log + 0.0005804260245392991 * tr_l2_rr + 0.0005338456556189634 * tr_l2_lassor + 0.03947761812798651 * tr_l2_HistReg + 0.00029262889516829937 * tr_l2_LR) 
print('OOF score: {}'.format(roc_auc_score(y_train.values, oof_blend)))

OOF score: 0.7666002003119106


In [156]:
sample_submission = pd.read_csv("D:\\COMPI-TOP\\cascade-cup-22\\sample_submission.csv")

In [157]:
sub_blend = (0.8535318456738976 * te_l2_log + 0.0005804260245392991 * te_l2_rr + 0.0005338456556189634 * te_l2_lassor + 0.03947761812798651 * te_l2_HistReg + 0.00029262889516829937 * te_l2_LR) 
sample_submission.cancelled = sub_blend
sample_submission.head()

,order_id,cancelled
0,130231,0.006940
1,130232,0.007129
2,130233,0.006625
3,130234,0.007099
4,130235,0.006610


In [158]:
sample_submission.to_csv("D:\\COMPI-TOP\\cascade-cup-22\\L2_Blend_Meta.csv", index=False)
sample_submission.head()

,order_id,cancelled
0,130231,0.006940
1,130232,0.007129
2,130233,0.006625
3,130234,0.007099
4,130235,0.006610


## *Checking the diff b/w public lb & CV to select best submission*

In [458]:
score_dict

{'Average Meta Blend': 0.00458979968808948,
 'L0_LGBMCLF': 0.0020684926732363884,
 'L0_LGBMCLF_TUNED': 0.0014770974729031439,
 'L1-Model: LogR': 0.0027410445579530984,
 'L1-Model: RidgeR': 0.005801141897702422,
 'L2-Model: LogR': 0.004455591470675202,
 'L2-Model: RidgeR': 0.005277689668866659,
 'L2-Model: LassoR': 0.005285927410675018,
 'L2-Model: HistGBREG': 0.0014781087906097223,
 'L2-Model: LinearREG': 0.005406327896585794,
 'Average L2': 0.0051176249365800786}

In [459]:
sorted_dict = dict(sorted(score_dict.items(), key=lambda item: item[1]))

In [460]:
sorted_dict

{'L0_LGBMCLF_TUNED': 0.0014770974729031439,
 'L2-Model: HistGBREG': 0.0014781087906097223,
 'L0_LGBMCLF': 0.0020684926732363884,
 'L1-Model: LogR': 0.0027410445579530984,
 'L2-Model: LogR': 0.004455591470675202,
 'Average Meta Blend': 0.00458979968808948,
 'Average L2': 0.0051176249365800786,
 'L2-Model: RidgeR': 0.005277689668866659,
 'L2-Model: LassoR': 0.005285927410675018,
 'L2-Model: LinearREG': 0.005406327896585794,
 'L1-Model: RidgeR': 0.005801141897702422}

# OPTUNAS

# L0 MODELS

## Lgbm-clf

In [462]:
def run(trial):
    fold = 0
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=500, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [463]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 23:21:27,760] A new study created in memory with name: no-name-83ff9f10-22a3-4772-81d0-b773eeb7a2a1
[I 2022-02-03 23:21:37,697] Trial 0 finished with value: 0.782317722744881 and parameters: {'learning_rate': 0.014593188532828101, 'lambda_l1': 1.2309450344540292e-05, 'lambda_l2': 0.005209005156413746, 'num_leaves': 133, 'feature_fraction': 0.9155328407461846, 'bagging_fraction': 0.6504575125283558, 'bagging_freq': 3, 'min_child_samples': 9}. Best is trial 0 with value: 0.782317722744881.
[I 2022-02-03 23:21:46,404] Trial 1 finished with value: 0.7878223574986166 and parameters: {'learning_rate': 0.019337647832064497, 'lambda_l1': 7.924774655944152e-06, 'lambda_l2': 0.6490527173061319, 'num_leaves': 31, 'feature_fraction': 0.6955358995484817, 'bagging_fraction': 0.9042447853300857, 'bagging_freq': 3, 'min_child_samples': 66}. Best is trial 1 with value: 0.7878223574986166.
[I 2022-02-03 23:21:56,484] Trial 2 finished with value: 0.7723242079413393 and parameters: {'learnin

[I 2022-02-03 23:24:24,894] Trial 20 finished with value: 0.7912280368013281 and parameters: {'learning_rate': 0.06319057715939608, 'lambda_l1': 1.309741410804374e-06, 'lambda_l2': 6.082090256108576e-08, 'num_leaves': 156, 'feature_fraction': 0.4473400351629755, 'bagging_fraction': 0.9309823812150928, 'bagging_freq': 6, 'min_child_samples': 96}. Best is trial 11 with value: 0.7986787493082456.
[I 2022-02-03 23:24:32,356] Trial 21 finished with value: 0.7825524003873825 and parameters: {'learning_rate': 0.06778640114480021, 'lambda_l1': 1.1578733756415979e-06, 'lambda_l2': 6.231236339214873e-08, 'num_leaves': 168, 'feature_fraction': 0.4366377119289849, 'bagging_fraction': 0.937122047471401, 'bagging_freq': 6, 'min_child_samples': 98}. Best is trial 11 with value: 0.7986787493082456.
[I 2022-02-03 23:24:41,039] Trial 22 finished with value: 0.7835168788046485 and parameters: {'learning_rate': 0.03959031599902426, 'lambda_l1': 5.691305462632434e-08, 'lambda_l2': 1.0594929343518959e-08, '

[I 2022-02-03 23:27:08,762] Trial 40 finished with value: 0.7867541159380189 and parameters: {'learning_rate': 0.07760434169067681, 'lambda_l1': 1.5208209170631646e-06, 'lambda_l2': 0.0005041248660315732, 'num_leaves': 75, 'feature_fraction': 0.5989626380318095, 'bagging_fraction': 0.6237968716666642, 'bagging_freq': 7, 'min_child_samples': 64}. Best is trial 11 with value: 0.7986787493082456.
[I 2022-02-03 23:27:13,609] Trial 41 finished with value: 0.7871600027670171 and parameters: {'learning_rate': 0.11465161784428113, 'lambda_l1': 3.0787204707997535e-07, 'lambda_l2': 2.5063033093509944e-08, 'num_leaves': 19, 'feature_fraction': 0.5321642832710635, 'bagging_fraction': 0.9697797631031012, 'bagging_freq': 7, 'min_child_samples': 23}. Best is trial 11 with value: 0.7986787493082456.
[I 2022-02-03 23:27:19,035] Trial 42 finished with value: 0.7877409034311013 and parameters: {'learning_rate': 0.07910780192968316, 'lambda_l1': 1.1422662396349559e-07, 'lambda_l2': 2.1198015863323896e-07,

In [464]:
study.best_params

{'learning_rate': 0.033393159236481285,
 'lambda_l1': 3.346640904962391e-08,
 'lambda_l2': 3.908099023297473e-08,
 'num_leaves': 16,
 'feature_fraction': 0.43480396855094344,
 'bagging_fraction': 0.9883525790842624,
 'bagging_freq': 5,
 'min_child_samples': 25}

In [465]:
study.best_value

0.7986787493082456

-------lgbm1--------
{'learning_rate': 0.04394129972546718,
 'lambda_l1': 3.332009828221393e-08,
 'lambda_l2': 2.8764308768784934,
 'num_leaves': 96,
 'feature_fraction': 0.5517245233128265,
 'bagging_fraction': 0.5205133833553873,
 'bagging_freq': 5,
 'min_child_samples': 27}
0.7955257332595463

-------lgbm1--------

-------lgbm2--------
{'learning_rate': 0.033393159236481285,
 'lambda_l1': 3.346640904962391e-08,
 'lambda_l2': 3.908099023297473e-08,
 'num_leaves': 16,
 'feature_fraction': 0.43480396855094344,
 'bagging_fraction': 0.9883525790842624,
 'bagging_freq': 5,
 'min_child_samples': 25}
 0.7986787493082456
 
-------lgbm2--------

In [449]:
%%time
scores = []
final_test_predictions = []
def run(trial):
    
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt"]),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
        'bagging_fraction': trial.suggest_categorical('bagging_fraction', [.4, .5, .45, 0.6, 0.7]),
        'feature_fraction': trial.suggest_categorical('feature_fraction', [.3, .4, .5, 0.6, 0.7, 0.80]),
        'learning_rate': trial.suggest_categorical('learning_rate', [.018, .0186, .0192, 0.0196, .0193, .0199]),
        'max_depth': trial.suggest_int('max_depth', 5, 20, step=1),
        'num_leaves' : trial.suggest_int('num_leaves', 650, 850, step=5),
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80, step=5),      
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 300, 600, step=20),
        "max_bin": trial.suggest_int("max_bin", 250, 350),
        'min_gain_to_split': trial.suggest_categorical('min_gain_to_split', [0.02084372652774491, 0.01084372652774491, 0.03084372652774491]),
        }
    
    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    return roc

Wall time: 2.77 ms


In [450]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 22:39:55,213] A new study created in memory with name: no-name-a24035ad-e85b-435f-8307-24c0189a5316
[I 2022-02-03 22:40:07,673] Trial 0 finished with value: 0.7683790467625901 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.05228118140761344, 'lambda_l2': 0.0019957367168903207, 'bagging_fraction': 0.5, 'feature_fraction': 0.4, 'learning_rate': 0.0196, 'max_depth': 18, 'num_leaves': 710, 'min_child_samples': 60, 'min_data_in_leaf': 360, 'max_bin': 339, 'min_gain_to_split': 0.02084372652774491}. Best is trial 0 with value: 0.7683790467625901.


ROC-AUC is:  0.7683790467625901
0 0.7683790467625901


[I 2022-02-03 22:40:16,599] Trial 1 finished with value: 0.7790516048699502 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 2.440130395485377, 'lambda_l2': 0.0024931094844332495, 'bagging_fraction': 0.7, 'feature_fraction': 0.8, 'learning_rate': 0.018, 'max_depth': 11, 'num_leaves': 770, 'min_child_samples': 55, 'min_data_in_leaf': 320, 'max_bin': 254, 'min_gain_to_split': 0.03084372652774491}. Best is trial 1 with value: 0.7790516048699502.


ROC-AUC is:  0.7790516048699502
0 0.7790516048699502


[I 2022-02-03 22:40:25,273] Trial 2 finished with value: 0.7841219908688433 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.0017648636827915717, 'lambda_l2': 1.4257020514359644, 'bagging_fraction': 0.7, 'feature_fraction': 0.7, 'learning_rate': 0.0192, 'max_depth': 8, 'num_leaves': 650, 'min_child_samples': 60, 'min_data_in_leaf': 580, 'max_bin': 336, 'min_gain_to_split': 0.02084372652774491}. Best is trial 2 with value: 0.7841219908688433.


...

ROC-AUC is:  0.7833157512451577
0 0.7833157512451577


[I 2022-02-03 22:47:01,675] Trial 48 finished with value: 0.7780727725511898 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.6293269161634566, 'lambda_l2': 0.3664284273762438, 'bagging_fraction': 0.45, 'feature_fraction': 0.7, 'learning_rate': 0.018, 'max_depth': 15, 'num_leaves': 805, 'min_child_samples': 70, 'min_data_in_leaf': 440, 'max_bin': 315, 'min_gain_to_split': 0.03084372652774491}. Best is trial 42 with value: 0.7944249792473712.


ROC-AUC is:  0.7780727725511898
0 0.7780727725511898


[I 2022-02-03 22:47:11,396] Trial 49 finished with value: 0.7858684975096846 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.2733440988165476, 'lambda_l2': 3.885566230343337, 'bagging_fraction': 0.6, 'feature_fraction': 0.8, 'learning_rate': 0.0199, 'max_depth': 9, 'num_leaves': 765, 'min_child_samples': 65, 'min_data_in_leaf': 480, 'max_bin': 337, 'min_gain_to_split': 0.01084372652774491}. Best is trial 42 with value: 0.7944249792473712.


ROC-AUC is:  0.7858684975096846
0 0.7858684975096846


In [451]:
study.best_params

{'boosting_type': 'gbdt',
 'lambda_l1': 2.013079655498328,
 'lambda_l2': 6.704712390093808,
 'bagging_fraction': 0.45,
 'feature_fraction': 0.6,
 'learning_rate': 0.0199,
 'max_depth': 10,
 'num_leaves': 790,
 'min_child_samples': 65,
 'min_data_in_leaf': 480,
 'max_bin': 336,
 'min_gain_to_split': 0.01084372652774491}

In [452]:
study.best_value

0.7944249792473712

-------lgbm1--------
{'boosting_type': 'gbdt',
 'lambda_l1': 0.5806043684908021,
 'lambda_l2': 7.106418698435042,
 'bagging_fraction': 0.45,
 'feature_fraction': 0.5,
 'learning_rate': 0.018,
 'max_depth': 8,
 'num_leaves': 775,
 'min_child_samples': 75,
 'min_data_in_leaf': 520,
 'max_bin': 331,
 'min_gain_to_split': 0.01084372652774491}
 0.7929055409518538
-------lgbm1--------
-------lgbm2--------
'boosting_type': 'gbdt',
 'lambda_l1': 0.004120454129529359,
 'lambda_l2': 0.3054014755871697,
 'bagging_fraction': 0.6,
 'feature_fraction': 0.6,
 'learning_rate': 0.0199,
 'max_depth': 7,
 'num_leaves': 685,
 'min_child_samples': 45,
 'min_data_in_leaf': 500,
 'max_bin': 288,
 'min_gain_to_split': 0.02084372652774491
0.7912446389042612
-------lgbm2--------
-------lgbm3--------
'boosting_type': 'gbdt',
 'lambda_l1': 2.013079655498328,
 'lambda_l2': 6.704712390093808,
 'bagging_fraction': 0.45,
 'feature_fraction': 0.6,
 'learning_rate': 0.0199,
 'max_depth': 10,
 'num_leaves': 790,
 'min_child_samples': 65,
 'min_data_in_leaf': 480,
 'max_bin': 336,
 'min_gain_to_split': 0.01084372652774491
study.best_value
-------lgbm3--------

## XGB-clf

In [401]:
%%time
scores = []
final_test_predictions = []
def run(trial):
    
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "metric": "auc",
        "boosting_type": "gbdt",
        "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1e-8, 100.0),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1e-8, 100.0),
        "subsample" : trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "max_depth" : trial.suggest_int("max_depth", 1, 7),
        "min_child_weight" : trial.suggest_int('min_child_weight', 1, 300),
        "n_estimators":5000
    }
    
    model = XGBClassifier(**param, random_state=RANDOM_SEED)
    model.fit(xtrain, ytrain,early_stopping_rounds=300,
              eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    return roc

Wall time: 0 ns


In [402]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 18:35:22,810] A new study created in memory with name: no-name-ccece788-5e8f-47ff-984e-30688587c2da
C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:35:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:35:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-02-03 18:37:23,459] Trial 0 finished with value: 0.7752431516325402 and parameters: {'learning_rate': 0.09668052401418388, 'reg_lambda': 0.6462073184143035, 'reg_alpha': 22.120735382469068, 'subsample': 0.44796261053964315, 'colsample_bytree': 0.55192664979347, 'max_depth': 6, 'min_child_weight': 94}. Best is trial 0 with value: 0.7752431516325402.


ROC-AUC is:  0.7752431516325402
0 0.7752431516325402
[18:37:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[I 2022-02-03 18:59:58,395] Trial 11 finished with value: 0.7737240592141671 and parameters: {'learning_rate': 0.01116949670725907, 'reg_lambda': 0.00020756485314975357, 'reg_alpha': 1.7645380567334508e-08, 'subsample': 0.9697026728031609, 'colsample_bytree': 0.9921095336476412, 'max_depth': 1, 'min_child_weight': 1}. Best is trial 9 with value: 0.784625069175429.


ROC-AUC is:  0.7737240592141671
0 0.7737240592141671
[18:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "metric" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\Sarkhel\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [403]:
study.best_params

{'learning_rate': 0.01098685543556639,
 'reg_lambda': 0.0015620122636105737,
 'reg_alpha': 0.1719093206502173,
 'subsample': 0.4997703785123122,
 'colsample_bytree': 0.9017632656309243,
 'max_depth': 6,
 'min_child_weight': 34}

## CatBoost CLF

In [411]:
%%time
scores = []
final_test_predictions = []
def run(trial):
    
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
    }
    
    model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC",
                               random_state=RANDOM_SEED, 
                               l2_leaf_reg=50,
                                border_count=64,**param)
    model.fit(xtrain, ytrain, verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    return roc

Wall time: 0 ns


In [412]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 20:29:13,516] A new study created in memory with name: no-name-9754c668-a182-40e8-9147-fe4f720b7ed7
C:\Users\Sarkhel\anaconda3\lib\site-packages\optuna\trial\_trial.py:772: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
[I 2022-02-03 20:29:18,717] Trial 0 finished with value: 0.7566117874930824 and parameters: {'iterations': 147, 'depth': 8, 'learning_rate': 0.016082576963111838, 'random_strength': 64, 'bagging_temperature': 0.053930669381946995, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.7566117874930824.


ROC-AUC is:  0.7566117874930824
0 0.7566117874930824


C:\Users\Sarkhel\anaconda3\lib\site-packages\optuna\trial\_trial.py:772: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
[I 2022-02-03 20:29:23,803] Trial 1 finished with value: 0.7788695005534035 and parameters: {'iterations': 185, 'depth': 6, 'learning_rate': 0.2045239919002807, 'random_strength': 68, 'bagging_temperature': 10.216660135999916, 'od_type': 'Iter'}. Best is trial 1 with value: 0.7788695005534035.


ROC-AUC is:  0.7788695005534035
0 0.7788695005534035


C:\Users\Sarkhel\anaconda3\lib\site-packages\optuna\trial\_trial.py:772: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
[I 2022-02-03 20:29:34,538] Trial 2 finished with value: 0.7861123408965136 and parameters: {'iterations': 266, 'depth': 5, 'learning_rate': 0.1533163416079515, 'random_strength': 96, 'bagging_temperature': 0.9517328269611176, 'od_type': 'Iter'}. Best is trial 2 with value: 0.7861123408965136.


...

ROC-AUC is:  0.7867387244050914
0 0.7867387244050914


C:\Users\Sarkhel\anaconda3\lib\site-packages\optuna\trial\_trial.py:772: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
[I 2022-02-03 20:36:33,735] Trial 48 finished with value: 0.7827566408411732 and parameters: {'iterations': 255, 'depth': 4, 'learning_rate': 0.06395251926477732, 'random_strength': 15, 'bagging_temperature': 88.4070035702315, 'od_type': 'Iter'}. Best is trial 23 with value: 0.7869268815716657.


ROC-AUC is:  0.7827566408411732
0 0.7827566408411732


C:\Users\Sarkhel\anaconda3\lib\site-packages\optuna\trial\_trial.py:772: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
[I 2022-02-03 20:36:41,753] Trial 49 finished with value: 0.782461434698395 and parameters: {'iterations': 202, 'depth': 4, 'learning_rate': 0.08065317628310259, 'random_strength': 23, 'bagging_temperature': 56.6701028257206, 'od_type': 'Iter'}. Best is trial 23 with value: 0.7869268815716657.


ROC-AUC is:  0.782461434698395
0 0.782461434698395


In [414]:
study.best_params

{'iterations': 260,
 'depth': 8,
 'learning_rate': 0.17572035142788606,
 'random_strength': 78,
 'bagging_temperature': 12.79633128785124,
 'od_type': 'Iter'}

In [415]:
study.best_value

0.7869268815716657

## LGBM Regressor

In [419]:
%%time
scores = []
final_test_predictions = []
def run(trial):
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    param = {
        "task": "train",
        "objective": "regression",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators":5000
    }

    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_val = lgb.Dataset(xvalid, yvalid)
    
    model = lgb.train(params=param,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=False)
   
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

Wall time: 1.99 ms


In [420]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 20:48:38,215] A new study created in memory with name: no-name-77091a5e-186a-4d6e-bc22-570b2a7aaeb6
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:48:42,411] Trial 0 finished with value: 0.7597064367736581 and parameters: {'learning_rate': 0.18412918974853412, 'lambda_l1': 1.1030867205789016e-08, 'lambda_l2': 1.2780032841046625e-05, 'num_leaves': 176, 'feature_fraction': 0.585685675064665, 'bagging_fraction': 0.6434823877931792, 'bagging_freq': 4, 'min_child_samples': 47}. Best is trial 0 with value: 0.7597064367736581.
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:48:45,8

C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:49:50,410] Trial 13 finished with value: 0.7882661178749308 and parameters: {'learning_rate': 0.06967164357708935, 'lambda_l1': 0.030076481363161284, 'lambda_l2': 1.335742819182237e-07, 'num_leaves': 58, 'feature_fraction': 0.4075581193305498, 'bagging_fraction': 0.8721695361104708, 'bagging_freq': 4, 'min_child_samples': 81}. Best is trial 10 with value: 0.7902784311012728.
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:49:53,491] Trial 14 finished with value: 0.7837813018815717 and parameters: {'learning_rate': 0.1122751480595531, 'lambda_

C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:50:59,684] Trial 26 finished with value: 0.785071077753182 and parameters: {'learning_rate': 0.03279407740466422, 'lambda_l1': 0.1953379310731571, 'lambda_l2': 3.7906852158908664e-07, 'num_leaves': 77, 'feature_fraction': 0.6221107125551192, 'bagging_fraction': 0.9131037814711634, 'bagging_freq': 1, 'min_child_samples': 16}. Best is trial 10 with value: 0.7902784311012728.
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:51:05,452] Trial 27 finished with value: 0.7870630706972883 and parameters: {'learning_rate': 0.0389642404571328, 'lambda_l1

C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:52:08,972] Trial 39 finished with value: 0.7807697495849474 and parameters: {'learning_rate': 0.16120253383679212, 'lambda_l1': 0.11277742937385415, 'lambda_l2': 6.850696768519903e-07, 'num_leaves': 94, 'feature_fraction': 0.9120450773207973, 'bagging_fraction': 0.88686095213242, 'bagging_freq': 7, 'min_child_samples': 62}. Best is trial 10 with value: 0.7902784311012728.
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2022-02-03 20:52:16,245] Trial 40 finished with value: 0.7846112340896514 and parameters: {'learning_rate': 0.04097269778288491, 'lambda_l1

In [422]:
study.best_params

{'learning_rate': 0.01243326765467829,
 'lambda_l1': 0.44583405781585317,
 'lambda_l2': 1.945205216342612e-07,
 'num_leaves': 135,
 'feature_fraction': 0.45977662840830263,
 'bagging_fraction': 0.5398639147101038,
 'bagging_freq': 4,
 'min_child_samples': 95}

In [423]:
study.best_value

0.7924220047039292

## CatBoost Regressor

In [433]:
%%time
scores = []
final_test_predictions = []
def run(trial):
    
    fold = 0
    
    
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    iterations = trial.suggest_int("iterations", 10, 2000)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 0.1, 15)
    depth = trial.suggest_int("depth", 1, 7)
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    xtest = xtest[features]
    
    model = CatBoostRegressor(
        random_state=RANDOM_SEED,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        depth=depth
       
    )
    
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict(xvalid)
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    return roc

Wall time: 0 ns


In [434]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-03 21:19:31,336] A new study created in memory with name: no-name-f2556b31-d52e-4b58-a1ab-d44799cb05f2
[I 2022-02-03 21:19:55,586] Trial 0 finished with value: 0.781883646928611 and parameters: {'learning_rate': 0.012067143008193212, 'iterations': 1666, 'l2_leaf_reg': 12.90198528594646, 'depth': 5}. Best is trial 0 with value: 0.781883646928611.


ROC-AUC is:  0.781883646928611
0 0.781883646928611


[I 2022-02-03 21:20:14,565] Trial 1 finished with value: 0.7744203099059214 and parameters: {'learning_rate': 0.01179087299773442, 'iterations': 866, 'l2_leaf_reg': 5.490995311385601, 'depth': 3}. Best is trial 0 with value: 0.781883646928611.


ROC-AUC is:  0.7744203099059214
0 0.7744203099059214


[I 2022-02-03 21:20:28,705] Trial 2 finished with value: 0.7821726272827892 and parameters: {'learning_rate': 0.07571501580991345, 'iterations': 825, 'l2_leaf_reg': 3.8260945001605444, 'depth': 7}. Best is trial 2 with value: 0.7821726272827892.


...

ROC-AUC is:  0.7856927919203098
0 0.7856927919203098


[I 2022-02-03 21:38:44,705] Trial 48 finished with value: 0.7854408204205866 and parameters: {'learning_rate': 0.026868257467843448, 'iterations': 1648, 'l2_leaf_reg': 9.23860372371242, 'depth': 7}. Best is trial 6 with value: 0.7872530437188711.


ROC-AUC is:  0.7854408204205866
0 0.7854408204205866


[I 2022-02-03 21:39:01,957] Trial 49 finished with value: 0.7725213579136689 and parameters: {'learning_rate': 0.01791714375677783, 'iterations': 1850, 'l2_leaf_reg': 8.205384919511634, 'depth': 2}. Best is trial 6 with value: 0.7872530437188711.


ROC-AUC is:  0.7725213579136689
0 0.7725213579136689


In [435]:
study.best_params

{'learning_rate': 0.023897097025582358,
 'iterations': 27,
 'l2_leaf_reg': 2.328838945916324,
 'depth': 6}

In [436]:
study.best_value

0.7872530437188711

# L1 MODELS

In [76]:
def run(trial):
    fold = 0
    fold = 0
    
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.cancelled
    yvalid = xvalid.cancelled
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    model = LGBMClassifier(**param, random_state=RANDOM_SEED, n_estimators=10000)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=False)
    preds_valid = model.predict_proba(xvalid)[:, 1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [77]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=50)

[I 2022-02-04 11:44:46,686] A new study created in memory with name: no-name-b8e63c12-6c24-4430-a9cb-4421c84945c2
[I 2022-02-04 11:44:50,842] Trial 0 finished with value: 0.7957666366906475 and parameters: {'learning_rate': 0.053379430813035124, 'lambda_l1': 6.418897085204742e-07, 'lambda_l2': 0.4158071340628894, 'num_leaves': 163, 'feature_fraction': 0.6435372688805586, 'bagging_fraction': 0.6196357775911578, 'bagging_freq': 2, 'min_child_samples': 48}. Best is trial 0 with value: 0.7957666366906475.
[I 2022-02-04 11:44:55,035] Trial 1 finished with value: 0.7972501902324295 and parameters: {'learning_rate': 0.04290869272261494, 'lambda_l1': 0.16787489832904565, 'lambda_l2': 3.397134234296255e-05, 'num_leaves': 131, 'feature_fraction': 0.5797161431870829, 'bagging_fraction': 0.8973462657391674, 'bagging_freq': 6, 'min_child_samples': 62}. Best is trial 1 with value: 0.7972501902324295.
[I 2022-02-04 11:45:01,227] Trial 2 finished with value: 0.7971932934421693 and parameters: {'learni

[I 2022-02-04 11:46:17,361] Trial 20 finished with value: 0.7958819002490315 and parameters: {'learning_rate': 0.010402502811160178, 'lambda_l1': 0.04641368632561649, 'lambda_l2': 1.4510105892423081e-08, 'num_leaves': 38, 'feature_fraction': 0.8792351772248292, 'bagging_fraction': 0.5020553286973695, 'bagging_freq': 1, 'min_child_samples': 47}. Best is trial 9 with value: 0.8032475269784172.
[I 2022-02-04 11:46:20,716] Trial 21 finished with value: 0.7977500691754289 and parameters: {'learning_rate': 0.016697199986168147, 'lambda_l1': 0.01606211446700274, 'lambda_l2': 5.148018700413077e-07, 'num_leaves': 65, 'feature_fraction': 0.7283919574265415, 'bagging_fraction': 0.587723200695882, 'bagging_freq': 1, 'min_child_samples': 41}. Best is trial 9 with value: 0.8032475269784172.
[I 2022-02-04 11:46:23,730] Trial 22 finished with value: 0.7943844251521859 and parameters: {'learning_rate': 0.021335525089433215, 'lambda_l1': 1.2801728580311573, 'lambda_l2': 0.00011488441541639743, 'num_leav

[I 2022-02-04 11:47:51,172] Trial 41 finished with value: 0.7970408480907581 and parameters: {'learning_rate': 0.01233371044336176, 'lambda_l1': 1.4134595949836013e-07, 'lambda_l2': 0.023387529062469634, 'num_leaves': 181, 'feature_fraction': 0.7467134949692495, 'bagging_fraction': 0.8677887601708701, 'bagging_freq': 1, 'min_child_samples': 12}. Best is trial 9 with value: 0.8032475269784172.
[I 2022-02-04 11:47:56,821] Trial 42 finished with value: 0.7966856322634199 and parameters: {'learning_rate': 0.011750187557065266, 'lambda_l1': 2.5573884120387534e-07, 'lambda_l2': 0.20214581353410108, 'num_leaves': 190, 'feature_fraction': 0.7612963988679631, 'bagging_fraction': 0.801380466956451, 'bagging_freq': 1, 'min_child_samples': 5}. Best is trial 9 with value: 0.8032475269784172.
[I 2022-02-04 11:48:01,909] Trial 43 finished with value: 0.7977515391532928 and parameters: {'learning_rate': 0.025508544318433815, 'lambda_l1': 1.4594193211360259e-08, 'lambda_l2': 0.0006707680751086381, 'num

In [79]:
study.best_params

{'learning_rate': 0.01393661332341988,
 'lambda_l1': 0.08659179008415999,
 'lambda_l2': 0.0006382635608284498,
 'num_leaves': 155,
 'feature_fraction': 0.6848869087952731,
 'bagging_fraction': 0.635236716686201,
 'bagging_freq': 3,
 'min_child_samples': 45}

In [80]:
study.best_value

0.8032475269784172

In [6]:
import pandas as pd
import streamlit as st
import pickle

def recommend(movie, df):
    movie_index = df[df['original_title'] == movie].index[0]
    distances = similarity[movie_index]
    # sorting array
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []

    for movies in movies_list:
        recommended_movies.append(df.iloc[movies[0]].original_title)

    return recommended_movies

movie_dict = pickle.load(open('D:\\Project\\Movie Recommendation System\\movies.pkl', 'rb'))
movies = pd.DataFrame(movie_dict)

similarity = pickle.load(open('D:\\Project\\Movie Recommendation System\\similarity.pkl', 'rb'))


selected_movie_name = 'Avatar'
recommend(selected_movie_name, movies)

['Fandango', 'Beerfest', 'Land Ho!', 'The Sure Thing', 'Duets']